In [1]:
from collections import OrderedDict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
from typing import Dict, List
from collections import OrderedDict
from models import ( 
    CatboostForecaster,
    TimeSeriesForecaster, 
    RandomForestForecaster, 
    GradientBoostingRegressorForecaster
)
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

import matplotlib.pyplot as plt
import pickle
from statsmodels.tsa.seasonal import seasonal_decompose
import pytz
from datetime import datetime

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/aayushhalgekar/miniconda3/envs/oxxo_pipeline/lib/python3.11/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [10]:
def add_inflation(new_df):
    values = [
        3.68, 3.37, 3.14, 3.13, 3.16, 3.2, 3.12, 3.1, 3.23, 3.33, 3.27, 3.18, 3.15, 3.01, 3.21, 3.25, 3.31, 3.43, 3.79, 3.75,
        3.82, 3.86, 3.84, 3.8, 3.62, 3.72, 3.86, 3.51, 2.85, 2.19, 1.97, 1.88, 1.97, 1.75, 1.49, 1.61, 1.6, 1.56, 1.51, 1.95,
        3.3, 3.63, 3.97, 4.44, 4.51, 4.58, 5.26, 5.62, 6.94, 8.01, 8.53, 9.23, 9.07, 9.67, 10.21, 10.84, 11.44, 12.22, 12.53,
        13.12, 13.25, 13.28, 13.34, 12.82, 12.36, 12.13, 11.78, 11.43,10.79
    ]

    # Create a list of dates from Jan 2018 to Aug 2023
    date_range = pd.date_range(start='2018-01-01', end='2023-09-30', freq='D')

    # Create a DataFrame

    inflation = []
    df = pd.DataFrame()
    df['date'] = date_range
    i = 0
    prev_month = df['date'][0].month
    # Populate the 'Price' column with values based on the corresponding month's value
    for date in df['date']:
        month = date.month
        if prev_month != month:
            prev_month = month
            i+=1
        inflation.append(values[i])
        # df.at[date, 'Price'] = value

    # Print the resulting DataFrame
    df['inflation'] = inflation
    # print(df)
    # df = df.set_index('date')
    df['date'] = pd.to_datetime(df['date'])
    new_df['date'] = pd.to_datetime(new_df['date'])
    # filtered2 = pd.merge(filtered2, df, how='left', left_index=True, right_index=True)
    merged = pd.merge(new_df, df, on='date', how='left')
    return merged


def get_filterd_dataset(product_name):

    new_df = pd.read_csv('historic_pilot_info_catalan.csv')
    # new_df = new_df.dropna(how='all')

    filtered = new_df[new_df['product_name']==product_name]
    filtered['order_submited_datetime'] = pd.to_datetime(filtered['order_submited_datetime'])
    return filtered

def aggregate_by_hour(df, time_bucket, has_competitor_price):
    """
    Aggregate a DataFrame by a specified time bucket (e.g., 24 for daily aggregation).
    
    Args:
        df (pd.DataFrame): Input DataFrame with a datetime field.
        time_bucket (int): Number of hours in each time bucket for aggregation.
    
    Returns:
        pd.DataFrame: Aggregated DataFrame with the datetime bucketed and values aggregated.
    """
    # Create a timezone object for Colombia (CST)
    colombia_tz = pytz.timezone('America/Bogota')
    
    # Convert the 'order_submited_datetime' column to Colombia timezone
    df['order_submited_datetime'] = df['order_submited_datetime'].dt.tz_localize(pytz.utc).dt.tz_convert(colombia_tz)
    
    df['order_submited_datetime'] = pd.to_datetime(df['order_submited_datetime'])
    df['hour'] = df['order_submited_datetime'].dt.hour
    df['date'] = df['order_submited_datetime'].dt.date
    df['month'] = df['order_submited_datetime'].dt.month
    df['day_of_week_n'] = df['order_submited_datetime'].dt.day_of_week
    df['day_of_month'] = df['order_submited_datetime'].dt.day
    df['year'] = df['order_submited_datetime'].dt.year
    agg_df = df.groupby(['date', 'hour', 'net_price']).agg({
    'quantity': 'sum',
    'cost': 'first',
    'avg_bench': 'first',
    'product_id': 'first',
    'product_name': 'first',
    'gross_price': 'first',
    'day_of_week_n': 'first',
    'day_of_month': 'first',
    # 'date': 'first'
    'year':'first',
    'month': 'first'

}).reset_index()

    min_date = pd.to_datetime(agg_df['date']).min()
    max_date = pd.to_datetime(agg_df['date']).max()

    date_range = pd.date_range(start=min_date, end=max_date, freq='D')
    hour_range = list(range(24))

    full_df = pd.DataFrame([(date, hour) for date in date_range for hour in hour_range], columns=['date', 'hour'])
    full_df['date'] = full_df['date'].dt.date.astype(str)  # Convert datetime to string to match df's 'date' format
    temp = agg_df[['date','hour']]
    temp_unique = temp.drop_duplicates()
    temp_unique['date'] = pd.to_datetime(temp_unique['date'])
    temp_unique['date'] = temp_unique['date'].dt.date.astype(str)
    # Identify missing rows
    merged_df = pd.merge(full_df, temp_unique, on=['date', 'hour'], how='left', indicator=True)
    missing_rows = merged_df[merged_df['_merge'] == 'left_only'][['date', 'hour']]
    # missing_rows.head(23)
    agg_df['date'] = pd.to_datetime(agg_df['date'])
    agg_df['date'] = agg_df['date'].dt.date.astype(str)
    merged_df = pd.merge(agg_df, missing_rows, on=['date', 'hour'], how='outer')
    cols_to_fill = [col for col in merged_df.columns if col not in ['quantity']]
    merged_df = merged_df.sort_values(['date','hour'])
    merged_df[cols_to_fill] = merged_df[cols_to_fill].fillna(method='ffill')
    merged_df['quantity'].fillna(0, inplace=True)
    
    
    return merged_df

def create_features(dataframe, has_competitor_price):
    def is_weekend(day):
        return 1 if day in [5, 6] else 0

    dataframe['if_weekend'] = dataframe['day_of_week_n'].apply(is_weekend)
    dataframe['quantity_lag1'] = dataframe['quantity'].shift(1)
    dataframe['quantity_lag2'] = dataframe['quantity'].shift(2)
    dataframe['net_price_lag1'] = dataframe['net_price'].shift(1)
    dataframe['net_price_lag2'] = dataframe['net_price'].shift(2)
    dataframe['gross_price_lag1'] = dataframe['gross_price'].shift(1)
    dataframe['gross_price_lag2'] = dataframe['gross_price'].shift(2)
    dataframe['cost_lag1'] = dataframe['cost'].shift(1)
    for day_num in range(0, 7):
        day_col_name = f'net_price_day_{day_num}'
        dataframe[day_col_name] = dataframe.apply(lambda row: row['net_price'] if row['day_of_week_n'] == day_num else 0, axis=1)


    if has_competitor_price:
        dataframe['diff'] = dataframe['net_price'] - dataframe['avg_bench']
        dataframe['diff_lag1'] = dataframe['diff'].shift(1)
        dataframe['diff_lag2'] = dataframe['diff'].shift(2)
        dataframe['diff_cross'] = dataframe['net_price'] - dataframe['diff_lag1']
        dataframe['diff_cross_lag1'] = dataframe['diff_cross'].shift(1)
        dataframe['diff_cross_lag2'] = dataframe['diff_cross'].shift(2)
        dataframe['avg_bench_lag1'] = dataframe['avg_bench'].shift(1)
        dataframe['avg_bench_lag2'] = dataframe['avg_bench'].shift(2)

    dataframe = add_inflation(dataframe)
    # dataframe.set_index('bucketed_datetime', inplace=True)
    # dataframe.index.freq = 'D' 
    result = seasonal_decompose(dataframe['quantity'], model='additive', period=7)
    dataframe['residual'] = result.resid
    dataframe['trend'] = result.trend
    dataframe['seasonal'] = result.seasonal
    dataframe['residual_lag2'] = dataframe['residual'].shift(2)
    dataframe['trend_lag2'] = dataframe['trend'].shift(2)
    dataframe['seasonal_lag2'] = dataframe['seasonal'].shift(2)

    # dataframe = dataframe.dropna()
    print('features ', dataframe.info())
    return dataframe

In [29]:
product_name = "Rabano Estándar Kg"
time_bucket=24
has_competitor_price=1


filtered = get_filterd_dataset(product_name)
# daily_aggregated_df = aggregate_by_hour(filtered, time_bucket=time_bucket, has_competitor_price=has_competitor_price)
# features = create_features(daily_aggregated_df, has_competitor_price)

filtered['diff'] = filtered['net_price']-filtered['cost'] 

filtered[filtered['diff']<0]

/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_30909/4117965702.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['order_submited_datetime'] = pd.to_datetime(filtered['order_submited_datetime'])


,region_code,order_submited_date,order_submited_datetime,parent_category,category,product_id,product_name,cost,benchmark,median_bench,avg_bench,max_bench,gross_price,net_price,quantity,benchmark_count,diff
735,BOG,2022-12-01,2022-12-01 19:24:00,Frutas & verduras,Verduras,353982459564085093,Rabano Estándar Kg,4025.8236,NaN,NaN,NaN,NaN,3120.0,3120.0,0.5,NaN,-905.8236
1049,BOG,2022-12-05,2022-12-05 20:21:00,Frutas & verduras,Verduras,353982459564085093,Rabano Estándar Kg,4183.2099,NaN,NaN,NaN,NaN,3120.0,3120.0,0.5,NaN,-1063.2099
1058,BOG,2022-12-06,2022-12-06 21:12:00,Frutas & verduras,Verduras,353982459564085093,Rabano Estándar Kg,4669.8207,NaN,NaN,NaN,NaN,3120.0,3120.0,3.0,NaN,-1549.8207
1101,BOG,2022-12-13,2022-12-13 19:44:00,Frutas & verduras,Verduras,353982459564085093,Rabano Estándar Kg,6183.5319,NaN,NaN,NaN,NaN,5500.0,5500.0,0.5,NaN,-683.5319
1107,BOG,2022-12-13,2022-12-13 22:42:00,Frutas & verduras,Verduras,353982459564085093,Rabano Estándar Kg,6183.5319,NaN,NaN,NaN,NaN,5500.0,5500.0,0.5,NaN,-683.5319
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315963,BOG,2022-12-04,2022-12-04 15:17:00,Frutas & verduras,Verduras,353982459564085093,Rabano Estándar Kg,4078.8938,NaN,NaN,NaN,NaN,3120.0,3120.0,4.0,NaN,-958.8938
315967,BOG,2022-12-05,2022-12-05 02:02:00,Frutas & verduras,Verduras,353982459564085093,Rabano Estándar Kg,4183.2099,NaN,NaN,NaN,NaN,3120.0,3120.0,0.5,NaN,-1063.2099
316120,BOG,2023-01-06,2023-01-06 01:36:00,Frutas & verduras,Verduras,353982459564085093,Rabano Estándar Kg,7100.0000,NaN,NaN,NaN,NaN,6880.0,6880.0,1.5,NaN,-220.0000
317829,BOG,2023-09-29,2023-09-29 15:59:00,Frutas & verduras,Verduras,353982459564085093,Rabano Estándar Kg,4594.1957,4000.0,4000.0,4000.0,4000.0,3840.0,3840.0,0.5,1.0,-754.1957


In [30]:
product_name = "Cebolla Cabezona Roja Sin Pelar Tamaño Mixto Desde 2Kg"
time_bucket=24
has_competitor_price=1


filtered = get_filterd_dataset(product_name)
# daily_aggregated_df = aggregate_by_hour(filtered, time_bucket=time_bucket, has_competitor_price=has_competitor_price)
# features = create_features(daily_aggregated_df, has_competitor_price)

filtered['diff'] = filtered['net_price']-filtered['cost'] 

filtered[filtered['diff']<0]

/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_30909/4117965702.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['order_submited_datetime'] = pd.to_datetime(filtered['order_submited_datetime'])


,region_code,order_submited_date,order_submited_datetime,parent_category,category,product_id,product_name,cost,benchmark,median_bench,avg_bench,max_bench,gross_price,net_price,quantity,benchmark_count,diff
705,BOG,2022-11-26,2022-11-26 13:25:00,Frutas & verduras,Verduras,7749572820280257375,Cebolla Cabezona Roja Sin Pelar Tamaño Mixto D...,2076.7607,2300.0,2300.0,2300.0,2300.0,2300.00,2070.0,10.0,1.0,-6.7607
737,BOG,2022-12-02,2022-12-02 00:52:00,Frutas & verduras,Verduras,7749572820280257375,Cebolla Cabezona Roja Sin Pelar Tamaño Mixto D...,2646.9577,3125.0,3125.0,3125.0,3125.0,2915.00,2623.5,4.0,1.0,-23.4577
885,BOG,2022-11-29,2022-11-29 14:27:00,Frutas & verduras,Verduras,7749572820280257375,Cebolla Cabezona Roja Sin Pelar Tamaño Mixto D...,2142.7190,2500.0,2500.0,2500.0,2500.0,2300.00,2070.0,10.0,1.0,-72.7190
886,BOG,2022-11-29,2022-11-29 22:45:00,Frutas & verduras,Verduras,7749572820280257375,Cebolla Cabezona Roja Sin Pelar Tamaño Mixto D...,2142.7190,2500.0,2500.0,2500.0,2500.0,2369.00,2132.1,2.0,1.0,-10.6190
888,BOG,2022-11-30,2022-11-30 00:24:00,Frutas & verduras,Verduras,7749572820280257375,Cebolla Cabezona Roja Sin Pelar Tamaño Mixto D...,2166.3498,2500.0,2500.0,2500.0,2500.0,2369.00,2132.1,4.0,1.0,-34.2498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318717,BOG,2023-09-24,2023-09-24 22:17:00,Frutas & verduras,Verduras,7749572820280257375,Cebolla Cabezona Roja Sin Pelar Tamaño Mixto D...,4619.4508,4166.0,4166.0,4166.0,4166.0,4318.90,4090.0,4.0,1.0,-529.4508
318722,BOG,2023-09-24,2023-09-24 16:46:00,Frutas & verduras,Verduras,7749572820280257375,Cebolla Cabezona Roja Sin Pelar Tamaño Mixto D...,4619.4508,4166.0,4166.0,4166.0,4166.0,4318.90,4090.0,4.0,1.0,-529.4508
318723,BOG,2023-09-25,2023-09-25 00:55:00,Frutas & verduras,Verduras,7749572820280257375,Cebolla Cabezona Roja Sin Pelar Tamaño Mixto D...,4619.4508,4167.0,4167.0,4167.0,4167.0,4318.90,4090.0,2.0,1.0,-529.4508
318730,BOG,2023-09-25,2023-09-25 20:47:00,Frutas & verduras,Verduras,7749572820280257375,Cebolla Cabezona Roja Sin Pelar Tamaño Mixto D...,4619.4508,4167.0,4167.0,4167.0,4167.0,4318.90,4090.0,5.0,1.0,-529.4508


In [38]:
product_name = "Cebolla Cabezona Roja Sin Pelar Tamaño Mixto Desde 2Kg"
time_bucket=24
has_competitor_price=1


filtered = get_filterd_dataset(product_name)
# daily_aggregated_df = aggregate_by_hour(filtered, time_bucket=time_bucket, has_competitor_price=has_competitor_price)
# features = create_features(daily_aggregated_df, has_competitor_price)

filtered['diff'] = filtered['net_price']-filtered['cost'] 

filtered

/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_30909/4117965702.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['order_submited_datetime'] = pd.to_datetime(filtered['order_submited_datetime'])


,region_code,order_submited_date,order_submited_datetime,parent_category,category,product_id,product_name,cost,benchmark,median_bench,avg_bench,max_bench,gross_price,net_price,quantity,benchmark_count,diff
1,BOG,2022-10-19,2022-10-19 01:15:00,Frutas & verduras,Verduras,7749572820280257375,Cebolla Cabezona Roja Sin Pelar Tamaño Mixto D...,2899.9498,2916.0,2916.0,2916.0,2916.0,3605.00,3352.65,2.0,1.0,452.7002
11,BOG,2022-10-23,2022-10-23 00:07:00,Frutas & verduras,Verduras,7749572820280257375,Cebolla Cabezona Roja Sin Pelar Tamaño Mixto D...,2899.9498,2916.0,2916.0,2916.0,2916.0,3481.00,3237.33,4.0,1.0,337.3802
14,BOG,2022-10-23,2022-10-23 17:04:00,Frutas & verduras,Verduras,7749572820280257375,Cebolla Cabezona Roja Sin Pelar Tamaño Mixto D...,2899.9498,2916.0,2916.0,2916.0,2916.0,3380.00,3143.40,9.3,1.0,243.4502
25,BOG,2022-10-25,2022-10-25 19:01:00,Frutas & verduras,Verduras,7749572820280257375,Cebolla Cabezona Roja Sin Pelar Tamaño Mixto D...,2899.9498,2812.0,2812.0,2812.0,2812.0,3605.00,3352.65,2.0,1.0,452.7002
28,BOG,2022-10-24,2022-10-24 20:04:00,Frutas & verduras,Verduras,7749572820280257375,Cebolla Cabezona Roja Sin Pelar Tamaño Mixto D...,2899.9498,2916.0,2916.0,2916.0,2916.0,3481.00,3237.33,2.0,1.0,337.3802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318717,BOG,2023-09-24,2023-09-24 22:17:00,Frutas & verduras,Verduras,7749572820280257375,Cebolla Cabezona Roja Sin Pelar Tamaño Mixto D...,4619.4508,4166.0,4166.0,4166.0,4166.0,4318.90,4090.00,4.0,1.0,-529.4508
318722,BOG,2023-09-24,2023-09-24 16:46:00,Frutas & verduras,Verduras,7749572820280257375,Cebolla Cabezona Roja Sin Pelar Tamaño Mixto D...,4619.4508,4166.0,4166.0,4166.0,4166.0,4318.90,4090.00,4.0,1.0,-529.4508
318723,BOG,2023-09-25,2023-09-25 00:55:00,Frutas & verduras,Verduras,7749572820280257375,Cebolla Cabezona Roja Sin Pelar Tamaño Mixto D...,4619.4508,4167.0,4167.0,4167.0,4167.0,4318.90,4090.00,2.0,1.0,-529.4508
318730,BOG,2023-09-25,2023-09-25 20:47:00,Frutas & verduras,Verduras,7749572820280257375,Cebolla Cabezona Roja Sin Pelar Tamaño Mixto D...,4619.4508,4167.0,4167.0,4167.0,4167.0,4318.90,4090.00,5.0,1.0,-529.4508


In [31]:
product_name = "Champiñón Blanco Paris 500 gr Bandeja"
time_bucket=24
has_competitor_price=1


filtered = get_filterd_dataset(product_name)
# daily_aggregated_df = aggregate_by_hour(filtered, time_bucket=time_bucket, has_competitor_price=has_competitor_price)
# features = create_features(daily_aggregated_df, has_competitor_price)

filtered['diff'] = filtered['net_price']-filtered['cost'] 

filtered[filtered['diff']<0]

/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_30909/4117965702.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['order_submited_datetime'] = pd.to_datetime(filtered['order_submited_datetime'])


,region_code,order_submited_date,order_submited_datetime,parent_category,category,product_id,product_name,cost,benchmark,median_bench,avg_bench,max_bench,gross_price,net_price,quantity,benchmark_count,diff
2239,BOG,2023-02-04,2023-02-04 13:28:00,Frutas & verduras,Verduras,-1073190491589771159,Champiñón Blanco Paris 500 gr Bandeja,11773.5654,NaN,NaN,NaN,NaN,4100.0,4100.0,6.0,NaN,-7673.5654
2241,BOG,2023-02-06,2023-02-06 12:41:00,Frutas & verduras,Verduras,-1073190491589771159,Champiñón Blanco Paris 500 gr Bandeja,11727.6770,NaN,NaN,NaN,NaN,4100.0,4100.0,1.0,NaN,-7627.6770
2322,BOG,2023-02-02,2023-02-02 18:39:00,Frutas & verduras,Verduras,-1073190491589771159,Champiñón Blanco Paris 500 gr Bandeja,12011.4484,NaN,NaN,NaN,NaN,4100.0,4100.0,1.0,NaN,-7911.4484
3508,BOG,2023-02-01,2023-02-01 23:17:00,Frutas & verduras,Verduras,-1073190491589771159,Champiñón Blanco Paris 500 gr Bandeja,11446.5607,NaN,NaN,NaN,NaN,2520.0,2520.0,2.0,NaN,-8926.5607
8548,BOG,2023-02-02,2023-02-02 20:19:00,Frutas & verduras,Verduras,-1073190491589771159,Champiñón Blanco Paris 500 gr Bandeja,12011.4484,NaN,NaN,NaN,NaN,4100.0,4100.0,2.0,NaN,-7911.4484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318655,BOG,2023-09-13,2023-09-13 14:42:00,Frutas & verduras,Verduras,-1073190491589771159,Champiñón Blanco Paris 500 gr Bandeja,12693.6006,13000.0,13000.0,13000.0,13000.0,12950.0,12302.5,1.0,1.0,-391.1006
318663,BOG,2023-09-15,2023-09-15 20:48:00,Frutas & verduras,Verduras,-1073190491589771159,Champiñón Blanco Paris 500 gr Bandeja,12667.2648,13000.0,13000.0,13000.0,13000.0,12740.0,11338.6,1.0,1.0,-1328.6648
318751,BOG,2023-09-28,2023-09-28 00:31:00,Frutas & verduras,Verduras,-1073190491589771159,Champiñón Blanco Paris 500 gr Bandeja,12548.0995,12000.0,12000.0,12000.0,12000.0,13040.0,11996.8,1.0,1.0,-551.2995
318752,BOG,2023-09-28,2023-09-28 01:22:00,Frutas & verduras,Verduras,-1073190491589771159,Champiñón Blanco Paris 500 gr Bandeja,12548.0995,12000.0,12000.0,12000.0,12000.0,13040.0,11996.8,2.0,1.0,-551.2995


In [42]:
product_name = "Papa Criolla Lavada Semi (Mediana) Kg"
time_bucket=24
has_competitor_price=1


filtered = get_filterd_dataset(product_name)
# daily_aggregated_df = aggregate_by_hour(filtered, time_bucket=time_bucket, has_competitor_price=has_competitor_price)
# features = create_features(daily_aggregated_df, has_competitor_price)

filtered['diff'] = filtered['net_price']-filtered['cost'] 
temp = filtered[filtered['order_submited_date']>'2022-12-31']
temp
# filtered[filtered['diff']<0]

/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_30909/4117965702.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['order_submited_datetime'] = pd.to_datetime(filtered['order_submited_datetime'])


,region_code,order_submited_date,order_submited_datetime,parent_category,category,product_id,product_name,cost,benchmark,median_bench,avg_bench,max_bench,gross_price,net_price,quantity,benchmark_count,diff
1504,BOG,2023-01-03,2023-01-03 20:55:00,Frutas & verduras,Tubérculos,-3386165639547488916,Papa Criolla Lavada Semi (Mediana) Kg,4471.4877,3720.0,4185.5,4185.5,4651.0,3650.0,3650.000,7.0,2.0,-821.4877
1550,BOG,2023-01-03,2023-01-03 19:34:00,Frutas & verduras,Tubérculos,-3386165639547488916,Papa Criolla Lavada Semi (Mediana) Kg,4471.4877,3720.0,4185.5,4185.5,4651.0,3650.0,3321.500,8.0,2.0,-1149.9877
1646,BOG,2023-01-06,2023-01-06 18:04:00,Frutas & verduras,Tubérculos,-3386165639547488916,Papa Criolla Lavada Semi (Mediana) Kg,3781.1716,4186.0,4186.0,4186.0,4186.0,4100.0,4100.000,5.0,1.0,318.8284
1664,BOG,2023-01-10,2023-01-10 18:27:00,Frutas & verduras,Tubérculos,-3386165639547488916,Papa Criolla Lavada Semi (Mediana) Kg,3725.4859,4418.0,4418.0,4418.0,4418.0,4223.0,4223.000,2.5,2.0,497.5141
1671,BOG,2023-01-05,2023-01-05 22:05:00,Frutas & verduras,Tubérculos,-3386165639547488916,Papa Criolla Lavada Semi (Mediana) Kg,3887.1529,3720.0,3720.0,3720.0,3720.0,3760.0,3760.000,2.5,1.0,-127.1529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318743,BOG,2023-09-27,2023-09-27 23:53:00,Frutas & verduras,Tubérculos,-3386165639547488916,Papa Criolla Lavada Semi (Mediana) Kg,3151.2621,3721.0,3721.0,3721.0,3721.0,3710.0,3609.832,2.5,1.0,458.5699
318744,BOG,2023-09-28,2023-09-28 02:00:00,Frutas & verduras,Tubérculos,-3386165639547488916,Papa Criolla Lavada Semi (Mediana) Kg,3141.4997,3256.0,3256.0,3256.0,3256.0,3570.0,3570.000,4.0,2.0,428.5003
318749,BOG,2023-09-28,2023-09-28 00:45:00,Frutas & verduras,Tubérculos,-3386165639547488916,Papa Criolla Lavada Semi (Mediana) Kg,3141.4997,3256.0,3256.0,3256.0,3256.0,3710.0,3710.000,2.5,2.0,568.5003
318753,BOG,2023-09-27,2023-09-27 22:54:00,Frutas & verduras,Tubérculos,-3386165639547488916,Papa Criolla Lavada Semi (Mediana) Kg,3151.2621,3721.0,3721.0,3721.0,3721.0,3710.0,3710.000,2.5,1.0,558.7379


In [37]:
product_name = "Cebolla Cabezona Roja Limpia Tamaño Mixto Kg (Tamaño 🏠)"
time_bucket=24
has_competitor_price=1


filtered = get_filterd_dataset(product_name)
# daily_aggregated_df = aggregate_by_hour(filtered, time_bucket=time_bucket, has_competitor_price=has_competitor_price)
# features = create_features(daily_aggregated_df, has_competitor_price)

filtered['diff'] = filtered['net_price']-filtered['cost'] 

filtered[filtered['diff']<0]

/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_30909/4117965702.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['order_submited_datetime'] = pd.to_datetime(filtered['order_submited_datetime'])


,region_code,order_submited_date,order_submited_datetime,parent_category,category,product_id,product_name,cost,benchmark,median_bench,avg_bench,max_bench,gross_price,net_price,quantity,benchmark_count,diff
5416,BOG,2023-07-05,2023-07-05 02:47:00,Frutas & verduras,Verduras,5533752035170569793,Cebolla Cabezona Roja Limpia Tamaño Mixto Kg (...,5611.5364,NaN,NaN,NaN,NaN,5040.00,5040.0,5.0,NaN,-571.5364
5444,BOG,2023-07-07,2023-07-07 19:07:00,Frutas & verduras,Verduras,5533752035170569793,Cebolla Cabezona Roja Limpia Tamaño Mixto Kg (...,5713.9831,NaN,NaN,NaN,NaN,5040.00,5040.0,15.0,NaN,-673.9831
6530,BOG,2023-07-27,2023-07-27 03:35:00,Frutas & verduras,Verduras,5533752035170569793,Cebolla Cabezona Roja Limpia Tamaño Mixto Kg (...,6968.9313,NaN,NaN,NaN,NaN,6550.00,6550.0,5.0,NaN,-418.9313
6532,BOG,2023-07-27,2023-07-27 17:17:00,Frutas & verduras,Verduras,5533752035170569793,Cebolla Cabezona Roja Limpia Tamaño Mixto Kg (...,6968.9313,NaN,NaN,NaN,NaN,6550.00,6550.0,5.0,NaN,-418.9313
8979,BOG,2023-08-09,2023-08-09 00:37:00,Frutas & verduras,Verduras,5533752035170569793,Cebolla Cabezona Roja Limpia Tamaño Mixto Kg (...,9730.2039,NaN,NaN,NaN,NaN,8520.00,8520.0,1.0,NaN,-1210.2039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318271,BOG,2023-09-18,2023-09-18 18:24:00,Frutas & verduras,Verduras,5533752035170569793,Cebolla Cabezona Roja Limpia Tamaño Mixto Kg (...,7617.6779,6500.0,6500.0,6500.0,6500.0,6367.48,6030.0,5.0,1.0,-1587.6779
318302,BOG,2023-09-24,2023-09-24 13:53:00,Frutas & verduras,Verduras,5533752035170569793,Cebolla Cabezona Roja Limpia Tamaño Mixto Kg (...,6761.9339,5500.0,5500.0,5500.0,5500.0,6620.91,6270.0,1.0,1.0,-491.9339
318349,BOG,2023-09-28,2023-09-28 23:36:00,Frutas & verduras,Verduras,5533752035170569793,Cebolla Cabezona Roja Limpia Tamaño Mixto Kg (...,6469.0456,5500.0,5500.0,5500.0,5500.0,5797.25,5490.0,1.0,1.0,-979.0456
318654,BOG,2023-09-13,2023-09-13 22:03:00,Frutas & verduras,Verduras,5533752035170569793,Cebolla Cabezona Roja Limpia Tamaño Mixto Kg (...,8104.1340,6500.0,6500.0,6500.0,6500.0,8004.22,7580.0,1.0,1.0,-524.1340


# CreateDaata

In [103]:
from collections import OrderedDict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
from typing import Dict, List
from collections import OrderedDict
from models import ( 
    CatboostForecaster,
    TimeSeriesForecaster, 
    RandomForestForecaster, 
    GradientBoostingRegressorForecaster,
    # LightGBMForecaster,
    # ProphetForecaster,
    XGBoostForecaster
)
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

import matplotlib.pyplot as plt
import pickle
from statsmodels.tsa.seasonal import seasonal_decompose
import pytz
from datetime import datetime
def get_filterd_dataset(product_name):

    new_df = pd.read_csv('historic_pilot_info_catalan.csv')
    # new_df = new_df.dropna(how='all')

    filtered = new_df[new_df['product_name']==product_name]
    filtered['order_submited_datetime'] = pd.to_datetime(filtered['order_submited_datetime'])
    return filtered

def aggregate_by_hour(df, time_bucket, has_competitor_price):
    """
    Aggregate a DataFrame by a specified time bucket (e.g., 24 for daily aggregation).
    
    Args:
        df (pd.DataFrame): Input DataFrame with a datetime field.
        time_bucket (int): Number of hours in each time bucket for aggregation.
    
    Returns:
        pd.DataFrame: Aggregated DataFrame with the datetime bucketed and values aggregated.
    """
    # Create a timezone object for Colombia (CST)
    colombia_tz = pytz.timezone('America/Bogota')
    
    # Convert the 'order_submited_datetime' column to Colombia timezone
    if df['order_submited_datetime'].dt.tz is None:
    # If the datetime is naive, localize it
        df['order_submited_datetime'] = df['order_submited_datetime'].dt.tz_localize(pytz.utc).dt.tz_convert(colombia_tz)
    else:
        # If the datetime is already timezone-aware, just convert it
        df['order_submited_datetime'] = df['order_submited_datetime'].dt.tz_convert(colombia_tz)

    df['order_submited_datetime'] = pd.to_datetime(df['order_submited_datetime'])
    df['hour'] = df['order_submited_datetime'].dt.hour
    df['date'] = df['order_submited_datetime'].dt.date
    df['month'] = df['order_submited_datetime'].dt.month
    df['day_of_week_n'] = df['order_submited_datetime'].dt.day_of_week
    df['day_of_month'] = df['order_submited_datetime'].dt.day
    df['year'] = df['order_submited_datetime'].dt.year
    agg_df = df.groupby(['date', 'hour', 'net_price']).agg({
    'quantity': 'sum',
    'cost': 'first',
    'avg_bench': 'first',
    'product_id': 'first',
    'product_name': 'first',
    'gross_price': 'first',
    'day_of_week_n': 'first',
    'day_of_month': 'first',
    # 'date': 'first'
    'year':'first',
    'month': 'first'

}).reset_index()

    min_date = pd.to_datetime(agg_df['date']).min()
    max_date = pd.to_datetime(agg_df['date']).max()

    date_range = pd.date_range(start=min_date, end=max_date, freq='D')
    hour_range = list(range(24))

    full_df = pd.DataFrame([(date, hour) for date in date_range for hour in hour_range], columns=['date', 'hour'])
    full_df['date'] = full_df['date'].dt.date.astype(str)  # Convert datetime to string to match df's 'date' format
    temp = agg_df[['date','hour']]
    temp_unique = temp.drop_duplicates()
    temp_unique['date'] = pd.to_datetime(temp_unique['date'])
    temp_unique['date'] = temp_unique['date'].dt.date.astype(str)
    # Identify missing rows
    merged_df = pd.merge(full_df, temp_unique, on=['date', 'hour'], how='left', indicator=True)
    missing_rows = merged_df[merged_df['_merge'] == 'left_only'][['date', 'hour']]
    # missing_rows.head(23)
    agg_df['date'] = pd.to_datetime(agg_df['date'])
    agg_df['date'] = agg_df['date'].dt.date.astype(str)
    merged_df = pd.merge(agg_df, missing_rows, on=['date', 'hour'], how='outer')
    cols_to_fill = [col for col in merged_df.columns if col not in ['quantity']]
    merged_df[cols_to_fill] = merged_df[cols_to_fill].fillna(method='ffill')
    merged_df['quantity'].fillna(0, inplace=True)

    # if not comp
    daily_quantity = merged_df.groupby('date').agg({'quantity': 'sum'}).reset_index()


    # #  if  comp
    # daily_quantity = merged_df.groupby('date').agg({'quantity': 'sum', 'net_price': 'mean','avg_bench':'mean'}).reset_index()

    # # Step 2: Calculate 7-Day Rolling Average with 1-day Lag
    daily_quantity['rolling_avg_quantity_7'] = (
        daily_quantity['quantity']
        .rolling(window=7)
        .mean()
        .shift(2)  # This adds the 1-day lag
    )

    daily_quantity['rolling_avg_quantity_30'] = (
        daily_quantity['quantity']
        .rolling(window=30)
        .mean()
        .shift(2)  # This adds the 1-day lag
    )

    daily_quantity['quantity_lag2'] = (
        daily_quantity['quantity']
        .shift(2)  # This adds the 1-day lag
    )

    # daily_quantity['daily_avg_diff'] = daily_quantity['net_price'] - daily_quantity['avg_bench']
    # daily_quantity['avg_bench_lag1'] = daily_quantity['avg_bench'].shift(1)
    # daily_quantity['diff_cross'] = daily_quantity['net_price'] - daily_quantity['avg_bench_lag1']

    # daily_quantity['daily_avg_diff_lag2'] = (
    #     daily_quantity['daily_avg_diff']
    #     .shift(2)
    #  # This adds the 1-day lag
    # )
    # daily_quantity['diff_cross_lag2'] = (
    #     daily_quantity['diff_cross']
    #     .shift(2)
    #  # This adds the 1-day lag
    # )

    merged_df = merged_df.merge(daily_quantity[['date', 'rolling_avg_quantity_30','rolling_avg_quantity_7','quantity_lag2']], on='date', how='left')
    merged_df = merged_df.sort_values(['date','hour'])
    
    return merged_df

def create_features(dataframe, has_competitor_price):
    def is_weekend(day):
        return 1 if day in [5, 6] else 0

    dataframe['if_weekend'] = dataframe['day_of_week_n'].apply(is_weekend)
    # dataframe['quantity_lag1'] = dataframe['quantity'].shift(1)
    # dataframe['quantity_lag2'] = dataframe['quantity'].shift(2)
    # dataframe['net_price_lag1'] = dataframe['net_price'].shift(1)
    # dataframe['net_price_lag2'] = dataframe['net_price'].shift(2)
    dataframe['gross_price_lag1'] = dataframe['gross_price'].shift(1)
    dataframe['gross_price_lag2'] = dataframe['gross_price'].shift(2)
    dataframe['cost_lag1'] = dataframe['cost'].shift(1)
    for day_num in range(0, 7):
        day_col_name = f'net_price_day_{day_num}'
        dataframe[day_col_name] = dataframe.apply(lambda row: row['net_price'] if row['day_of_week_n'] == day_num else 0, axis=1)


    # if has_competitor_price:
        # dataframe['diff'] = dataframe['net_price'] - dataframe['avg_bench']
        # dataframe['diff_lag1'] = dataframe['diff'].shift(1)
        # dataframe['diff_lag2'] = dataframe['diff'].shift(2)
        # dataframe['diff_cross'] = dataframe['net_price'] - dataframe['diff_lag1']
        # dataframe['diff_cross_lag1'] = dataframe['diff_cross'].shift(1)
        # dataframe['diff_cross_lag2'] = dataframe['diff_cross'].shift(2)
        # dataframe['avg_bench_lag1'] = dataframe['avg_bench'].shift(1)
        # dataframe['avg_bench_lag2'] = dataframe['avg_bench'].shift(2)

    dataframe = add_inflation(dataframe)
    # dataframe.set_index('bucketed_datetime', inplace=True)
    # dataframe.index.freq = 'D' 
    result = seasonal_decompose(dataframe['quantity'], model='additive', period=7)
    dataframe['residual'] = result.resid
    dataframe['trend'] = result.trend
    dataframe['seasonal'] = result.seasonal
    dataframe['residual_lag2'] = dataframe['residual'].shift(2)
    dataframe['trend_lag2'] = dataframe['trend'].shift(2)
    dataframe['seasonal_lag2'] = dataframe['seasonal'].shift(2)

    # dataframe = dataframe.dropna()
    print('features ', dataframe.info())
    return dataframe


In [106]:
# product_name = "Papa Criolla Lavada Semi (Mediana) Kg"
time_bucket=24
has_competitor_price=1


filtered1 = get_filterd_dataset('Papa Sabanera Gruesa (Grande) Kg')
# filterd2 = get_filterd_dataset('Papa Sabanera Gruesa (Grande) Kg')

daily_aggregated_df = aggregate_by_hour(filtered1, time_bucket=time_bucket, has_competitor_price=has_competitor_price)
# features = create_features(daily_aggregated_df, has_competitor_price)
# features[features['date']=='2021-09-29']
daily_aggregated_df.tail(54)

/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_30909/3257728416.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['order_submited_datetime'] = pd.to_datetime(filtered['order_submited_datetime'])
/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_30909/3257728416.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_unique['date'] = pd.to_datetime(temp_unique['date'])
/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_30909/3257728416.py:88: SettingWithCopyWarning:

,date,hour,net_price,quantity,cost,avg_bench,product_id,product_name,gross_price,day_of_week_n,day_of_month,year,month,rolling_avg_quantity_30,rolling_avg_quantity_7,quantity_lag2
2120,2023-09-27,18,4090.0,2.5,4064.5604,4583.0,8.673177e+18,Papa Sabanera Gruesa (Grande) Kg,4090.0,2.0,27.0,2023.0,9.0,43.731667,46.064286,20.00
17531,2023-09-27,19,4400.0,0.0,4059.4123,4479.0,8.673177e+18,Papa Sabanera Gruesa (Grande) Kg,4400.0,4.0,29.0,2023.0,9.0,43.731667,46.064286,20.00
2121,2023-09-27,20,4470.0,15.0,4103.3104,4583.0,8.673177e+18,Papa Sabanera Gruesa (Grande) Kg,4470.0,2.0,27.0,2023.0,9.0,43.731667,46.064286,20.00
17532,2023-09-27,21,4400.0,0.0,4059.4123,4479.0,8.673177e+18,Papa Sabanera Gruesa (Grande) Kg,4400.0,4.0,29.0,2023.0,9.0,43.731667,46.064286,20.00
17533,2023-09-27,22,4400.0,0.0,4059.4123,4479.0,8.673177e+18,Papa Sabanera Gruesa (Grande) Kg,4400.0,4.0,29.0,2023.0,9.0,43.731667,46.064286,20.00
17534,2023-09-27,23,4400.0,0.0,4059.4123,4479.0,8.673177e+18,Papa Sabanera Gruesa (Grande) Kg,4400.0,4.0,29.0,2023.0,9.0,43.731667,46.064286,20.00
17535,2023-09-28,0,4400.0,0.0,4059.4123,4479.0,8.673177e+18,Papa Sabanera Gruesa (Grande) Kg,4400.0,4.0,29.0,2023.0,9.0,46.231333,47.848571,84.99
17536,2023-09-28,1,4400.0,0.0,4059.4123,4479.0,8.673177e+18,Papa Sabanera Gruesa (Grande) Kg,4400.0,4.0,29.0,2023.0,9.0,46.231333,47.848571,84.99
17537,2023-09-28,2,4400.0,0.0,4059.4123,4479.0,8.673177e+18,Papa Sabanera Gruesa (Grande) Kg,4400.0,4.0,29.0,2023.0,9.0,46.231333,47.848571,84.99
17538,2023-09-28,3,4400.0,0.0,4059.4123,4479.0,8.673177e+18,Papa Sabanera Gruesa (Grande) Kg,4400.0,4.0,29.0,2023.0,9.0,46.231333,47.848571,84.99


In [99]:
# x = filtered1[filtered1['date']=='2023-09-29'].sort_values('hour')
# x[x['hour']==19].info()
# x

,region_code,order_submited_date,order_submited_datetime,parent_category,category,product_id,product_name,cost,benchmark,median_bench,...,gross_price,net_price,quantity,benchmark_count,hour,date,month,day_of_week_n,day_of_month,year


In [113]:
df = get_filterd_dataset('Papa Sabanera Gruesa (Grande) Kg')
colombia_tz = pytz.timezone('America/Bogota')

# Convert the 'order_submited_datetime' column to Colombia timezone
if df['order_submited_datetime'].dt.tz is None:
# If the datetime is naive, localize it
    df['order_submited_datetime'] = df['order_submited_datetime'].dt.tz_localize(pytz.utc).dt.tz_convert(colombia_tz)
else:
    # If the datetime is already timezone-aware, just convert it
    df['order_submited_datetime'] = df['order_submited_datetime'].dt.tz_convert(colombia_tz)

df['order_submited_datetime'] = pd.to_datetime(df['order_submited_datetime'])
df['hour'] = df['order_submited_datetime'].dt.hour
df['date'] = df['order_submited_datetime'].dt.date
df['month'] = df['order_submited_datetime'].dt.month
df['day_of_week_n'] = df['order_submited_datetime'].dt.day_of_week
df['day_of_month'] = df['order_submited_datetime'].dt.day
df['year'] = df['order_submited_datetime'].dt.year
agg_df = df.groupby(['date', 'hour', 'net_price']).agg({
'quantity': 'sum',
'cost': 'first',
'avg_bench': 'first',
'product_id': 'first',
'product_name': 'first',
'gross_price': 'first',
'day_of_week_n': 'first',
'day_of_month': 'first',
# 'date': 'first'
'year':'first',
'month': 'first'

}).reset_index()

min_date = pd.to_datetime(agg_df['date']).min()
max_date = pd.to_datetime(agg_df['date']).max()

date_range = pd.date_range(start=min_date, end=max_date, freq='D')
hour_range = list(range(24))

full_df = pd.DataFrame([(date, hour) for date in date_range for hour in hour_range], columns=['date', 'hour'])
full_df['date'] = full_df['date'].dt.date.astype(str)  # Convert datetime to string to match df's 'date' format
temp = agg_df[['date','hour']]
temp_unique = temp.drop_duplicates()
temp_unique['date'] = pd.to_datetime(temp_unique['date'])
temp_unique['date'] = temp_unique['date'].dt.date.astype(str)
# Identify missing rows
merged_df = pd.merge(full_df, temp_unique, on=['date', 'hour'], how='left', indicator=True)
missing_rows = merged_df[merged_df['_merge'] == 'left_only'][['date', 'hour']]
# missing_rows.head(23)
agg_df['date'] = pd.to_datetime(agg_df['date'])
agg_df['date'] = agg_df['date'].dt.date.astype(str)
merged_df = pd.merge(agg_df, missing_rows, on=['date', 'hour'], how='outer')
cols_to_fill = [col for col in merged_df.columns if col not in ['quantity']]
merged_df[cols_to_fill] = merged_df[cols_to_fill].fillna(method='ffill')
merged_df['quantity'].fillna(0, inplace=True)

# if not comp
daily_quantity_papagrande = merged_df.groupby('date').agg({'quantity': 'sum','net_price': 'mean'}).reset_index()
daily_quantity_papagrande

daily_quantity_papagrande['papagrande_quantity_lag2'] = (
        daily_quantity_papagrande['quantity']
        .shift(2)  # This adds the 1-day lag
    )
daily_quantity_papagrande['papagrande_net_price_lag2'] = (
        daily_quantity_papagrande['net_price']
        .shift(2)  # This adds the 1-day lag
    )
daily_quantity_papagrande


/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_30909/3257728416.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['order_submited_datetime'] = pd.to_datetime(filtered['order_submited_datetime'])
/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_30909/3564863447.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_unique['date'] = pd.to_datetime(temp_unique['date'])
/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_30909/3564863447.py:45: SettingWithCopyWarning:

,date,quantity,net_price,papagrande_quantity_lag2,papagrande_net_price_lag2
0,2021-09-29,17.50,4272.500000,NaN,NaN
1,2021-09-30,50.00,4187.500000,NaN,NaN
2,2021-10-01,30.00,4181.666667,17.50,4272.500000
3,2021-10-02,12.50,4345.416667,50.00,4187.500000
4,2021-10-03,45.00,4072.500000,30.00,4181.666667
...,...,...,...,...,...
726,2023-09-25,20.00,4361.666667,64.49,4324.838173
727,2023-09-26,84.99,4252.001963,42.50,4295.000000
728,2023-09-27,55.00,4366.666667,20.00,4361.666667
729,2023-09-28,97.50,4361.250000,84.99,4252.001963


In [161]:
def aggregate_by_hour(df, time_bucket, has_competitor_price):
    """
    Aggregate a DataFrame by a specified time bucket (e.g., 24 for daily aggregation).
    
    Args:
        df (pd.DataFrame): Input DataFrame with a datetime field.
        time_bucket (int): Number of hours in each time bucket for aggregation.
    
    Returns:
        pd.DataFrame: Aggregated DataFrame with the datetime bucketed and values aggregated.
    """
    # Create a timezone object for Colombia (CST)
    colombia_tz = pytz.timezone('America/Bogota')
    
    # Convert the 'order_submited_datetime' column to Colombia timezone
    df['order_submited_datetime'] = df['order_submited_datetime'].dt.tz_localize(pytz.utc).dt.tz_convert(colombia_tz)
    
    df['order_submited_datetime'] = pd.to_datetime(df['order_submited_datetime'])
    df['hour'] = df['order_submited_datetime'].dt.hour
    df['date'] = df['order_submited_datetime'].dt.date
    df['month'] = df['order_submited_datetime'].dt.month
    df['day_of_week_n'] = df['order_submited_datetime'].dt.day_of_week
    df['day_of_month'] = df['order_submited_datetime'].dt.day
    df['year'] = df['order_submited_datetime'].dt.year
    agg_df = df.groupby(['date', 'hour', 'net_price']).agg({
    'quantity': 'sum',
    'cost': 'first',
    'avg_bench': 'first',
    'product_id': 'first',
    'product_name': 'first',
    'gross_price': 'first',
    'day_of_week_n': 'first',
    'day_of_month': 'first',
    # 'date': 'first'
    'year':'first',
    'month': 'first'

}).reset_index()

    min_date = pd.to_datetime(agg_df['date']).min()
    max_date = pd.to_datetime(agg_df['date']).max()

    date_range = pd.date_range(start=min_date, end=max_date, freq='D')
    hour_range = list(range(24))

    full_df = pd.DataFrame([(date, hour) for date in date_range for hour in hour_range], columns=['date', 'hour'])
    full_df['date'] = full_df['date'].dt.date.astype(str)  # Convert datetime to string to match df's 'date' format
    temp = agg_df[['date','hour']]
    temp_unique = temp.drop_duplicates()
    temp_unique['date'] = pd.to_datetime(temp_unique['date'])
    temp_unique['date'] = temp_unique['date'].dt.date.astype(str)
    # Identify missing rows
    merged_df = pd.merge(full_df, temp_unique, on=['date', 'hour'], how='left', indicator=True)
    missing_rows = merged_df[merged_df['_merge'] == 'left_only'][['date', 'hour']]
    # missing_rows.head(23)
    agg_df['date'] = pd.to_datetime(agg_df['date'])
    agg_df['date'] = agg_df['date'].dt.date.astype(str)
    merged_df = pd.merge(agg_df, missing_rows, on=['date', 'hour'], how='outer')
    cols_to_fill = [col for col in merged_df.columns if col not in ['quantity']]
    merged_df[cols_to_fill] = merged_df[cols_to_fill].fillna(method='ffill')
    merged_df['quantity'].fillna(0, inplace=True)

    # if not comp
    # daily_quantity = merged_df.groupby('date').agg({'quantity': 'sum', 'net_price': 'mean'}).reset_index()


    #  if  comp
    daily_quantity = merged_df.groupby('date').agg({'quantity': 'sum', 'net_price': 'mean','avg_bench':'mean'}).reset_index()

    # Step 2: Calculate 7-Day Rolling Average with 1-day Lag
    daily_quantity['rolling_avg_quantity_7'] = (
        daily_quantity['quantity']
        .rolling(window=7)
        .mean()
        .shift(2)  # This adds the 1-day lag
    )

    daily_quantity['rolling_avg_quantity_30'] = (
        daily_quantity['quantity']
        .rolling(window=30)
        .mean()
        .shift(2)  # This adds the 1-day lag
    )

    daily_quantity['quantity_lag2'] = (
        daily_quantity['quantity']
        .shift(2)  # This adds the 1-day lag
    )

    daily_quantity['diff'] = daily_quantity['net_price'] - daily_quantity['avg_bench']
    daily_quantity['avg_bench_lag1'] = daily_quantity['avg_bench'].shift(1)
    daily_quantity['diff_cross'] = daily_quantity['net_price'] - daily_quantity['avg_bench_lag1']

    daily_quantity['diff_lag2'] = (
        daily_quantity['diff']
        .shift(2)
     # This adds the 1-day lag
    )
    daily_quantity['diff_cross_lag2'] = (
        daily_quantity['diff_cross']
        .shift(2)
     # This adds the 1-day lag
    )

    merged_df = merged_df.merge(daily_quantity[['date', 'rolling_avg_quantity_30','rolling_avg_quantity_7','quantity_lag2','diff_lag2','diff_cross_lag2']], on='date', how='left')
    
    # daily_quantity_papagrande = get_papa_grande()
    # merged_df = merged_df.merge(daily_quantity_papagrande[['date','papagrande_net_price_lag2', 'papagrande_quantity_lag2']], on='date', how='left')

    merged_df = merged_df.sort_values(['date','hour'])
    
    return merged_df


In [164]:
# product_name = "Papa Criolla Lavada Semi (Mediana) Kg"
time_bucket=24
has_competitor_price=1


filtered1 = get_filterd_dataset('Papa Criolla Lavada Semi (Mediana) Kg')
# filterd2 = get_filterd_dataset('Papa Sabanera Gruesa (Grande) Kg')

daily_aggregated_df = aggregate_by_hour(filtered1, time_bucket=time_bucket, has_competitor_price=has_competitor_price)
# features = create_features(daily_aggregated_df, has_competitor_price)
# features[features['date']=='2021-09-29']
daily_aggregated_df.tail(50)

# merged_df = daily_aggregated_df.merge(daily_quantity_papagrande[['date','papagrande_net_price_lag2', 'papagrande_quantity_lag2']], on='date', how='left')
# merged_df = merged_df.sort_values(['date','hour'])
    
# merged_df.info()
# features = create_features(daily_aggregated_df, has_competitor_price)
# features

/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_30909/3257728416.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['order_submited_datetime'] = pd.to_datetime(filtered['order_submited_datetime'])
/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_30909/819295869.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_unique['date'] = pd.to_datetime(temp_unique['date'])
/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_30909/819295869.py:51: SettingWithCopyWarning: 


,date,hour,net_price,quantity,cost,avg_bench,product_id,product_name,gross_price,day_of_week_n,day_of_month,year,month,rolling_avg_quantity_30,rolling_avg_quantity_7,quantity_lag2,diff_lag2,diff_cross_lag2
21167,2023-09-28,16,3148.8320,2.5,3141.4997,3256.0,-3.386166e+18,Papa Criolla Lavada Semi (Mediana) Kg,3260.0,3.0,28.0,2023.0,9.0,396.842667,420.968571,491.60,-70.512121,-80.716799
21168,2023-09-28,16,3171.9800,7.5,3141.4997,3256.0,-3.386166e+18,Papa Criolla Lavada Semi (Mediana) Kg,3260.0,3.0,28.0,2023.0,9.0,396.842667,420.968571,491.60,-70.512121,-80.716799
21169,2023-09-28,16,3260.0000,35.0,3141.4997,3256.0,-3.386166e+18,Papa Criolla Lavada Semi (Mediana) Kg,3260.0,3.0,28.0,2023.0,9.0,396.842667,420.968571,491.60,-70.512121,-80.716799
21170,2023-09-28,16,3583.4900,5.0,3141.4997,3256.0,-3.386166e+18,Papa Criolla Lavada Semi (Mediana) Kg,3710.0,3.0,28.0,2023.0,9.0,396.842667,420.968571,491.60,-70.512121,-80.716799
21171,2023-09-28,17,3130.0000,18.0,3141.4997,3256.0,-3.386166e+18,Papa Criolla Lavada Semi (Mediana) Kg,3130.0,3.0,28.0,2023.0,9.0,396.842667,420.968571,491.60,-70.512121,-80.716799
21172,2023-09-28,17,3148.8293,7.5,3141.4997,3256.0,-3.386166e+18,Papa Criolla Lavada Semi (Mediana) Kg,3260.0,3.0,28.0,2023.0,9.0,396.842667,420.968571,491.60,-70.512121,-80.716799
21173,2023-09-28,17,3148.8320,10.0,3141.4997,3256.0,-3.386166e+18,Papa Criolla Lavada Semi (Mediana) Kg,3260.0,3.0,28.0,2023.0,9.0,396.842667,420.968571,491.60,-70.512121,-80.716799
21174,2023-09-28,17,3448.2600,4.0,3141.4997,3256.0,-3.386166e+18,Papa Criolla Lavada Semi (Mediana) Kg,3570.0,3.0,28.0,2023.0,9.0,396.842667,420.968571,491.60,-70.512121,-80.716799
28542,2023-09-28,18,3710.0000,0.0,3723.2401,3372.0,-3.386166e+18,Papa Criolla Lavada Semi (Mediana) Kg,3710.0,4.0,29.0,2023.0,9.0,396.842667,420.968571,491.60,-70.512121,-80.716799
21175,2023-09-28,19,3260.0000,2.5,3723.2401,3372.0,-3.386166e+18,Papa Criolla Lavada Semi (Mediana) Kg,3260.0,3.0,28.0,2023.0,9.0,396.842667,420.968571,491.60,-70.512121,-80.716799


In [154]:
time_bucket=24
has_competitor_price=1


filtered1 = get_filterd_dataset('Papa Sabanera Semi (Mediana) Kg')
# filterd2 = get_filterd_dataset('Papa Sabanera Gruesa (Grande) Kg')

daily_aggregated_df = aggregate_by_hour_High_Selling(filtered1, time_bucket=time_bucket, has_competitor_price=has_competitor_price)
# features = create_features(daily_aggregated_df, has_competitor_price)
# features[features['date']=='2021-09-29']
daily_aggregated_df[daily_aggregated_df['date']=='2023-09-28']

/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_30909/3257728416.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['order_submited_datetime'] = pd.to_datetime(filtered['order_submited_datetime'])


merged columns Index(['region_code', 'order_submited_date', 'order_submited_datetime',
       'parent_category', 'category', 'product_id', 'product_name', 'cost',
       'benchmark', 'median_bench', 'avg_bench', 'max_bench', 'gross_price',
       'net_price_x', 'quantity', 'benchmark_count', 'hour', 'date', 'month',
       'day_of_week_n', 'day_of_month', 'year', 'net_price_y',
       'max_net_price'],
      dtype='object')


,date,hour,max_net_price,quantity,cost,avg_bench,product_id,product_name,gross_price,day_of_week_n,day_of_month,year,month,net_price,rolling_avg_quantity_30,rolling_avg_quantity_7,quantity_lag2
17504,2023-09-28,0,4030.0,0.0,3835.1849,4479.0,-3.682187e+17,Papa Sabanera Semi (Mediana) Kg,4030.0,4.0,29.0,2023.0,9.0,4030.0,42.716667,55.214286,84.0
17505,2023-09-28,1,4030.0,0.0,3835.1849,4479.0,-3.682187e+17,Papa Sabanera Semi (Mediana) Kg,4030.0,4.0,29.0,2023.0,9.0,4030.0,42.716667,55.214286,84.0
17506,2023-09-28,2,4030.0,0.0,3835.1849,4479.0,-3.682187e+17,Papa Sabanera Semi (Mediana) Kg,4030.0,4.0,29.0,2023.0,9.0,4030.0,42.716667,55.214286,84.0
17507,2023-09-28,3,4030.0,0.0,3835.1849,4479.0,-3.682187e+17,Papa Sabanera Semi (Mediana) Kg,4030.0,4.0,29.0,2023.0,9.0,4030.0,42.716667,55.214286,84.0
17508,2023-09-28,4,4030.0,0.0,3835.1849,4479.0,-3.682187e+17,Papa Sabanera Semi (Mediana) Kg,4030.0,4.0,29.0,2023.0,9.0,4030.0,42.716667,55.214286,84.0
17509,2023-09-28,5,4030.0,0.0,3835.1849,4479.0,-3.682187e+17,Papa Sabanera Semi (Mediana) Kg,4030.0,4.0,29.0,2023.0,9.0,4030.0,42.716667,55.214286,84.0
17510,2023-09-28,6,4030.0,0.0,3835.1849,4479.0,-3.682187e+17,Papa Sabanera Semi (Mediana) Kg,4030.0,4.0,29.0,2023.0,9.0,4030.0,42.716667,55.214286,84.0
17511,2023-09-28,7,4030.0,0.0,3835.1849,4479.0,-3.682187e+17,Papa Sabanera Semi (Mediana) Kg,4030.0,4.0,29.0,2023.0,9.0,4030.0,42.716667,55.214286,84.0
17512,2023-09-28,8,4030.0,0.0,3835.1849,4479.0,-3.682187e+17,Papa Sabanera Semi (Mediana) Kg,4030.0,4.0,29.0,2023.0,9.0,4030.0,42.716667,55.214286,84.0
17513,2023-09-28,9,4030.0,0.0,3835.1849,4479.0,-3.682187e+17,Papa Sabanera Semi (Mediana) Kg,4030.0,4.0,29.0,2023.0,9.0,4030.0,42.716667,55.214286,84.0


In [160]:
import datetime

# Corrected code:
filtered1[filtered1['date'] == datetime.date(2023, 9, 28)].sort_values('hour')


,region_code,order_submited_date,order_submited_datetime,parent_category,category,product_id,product_name,cost,benchmark,median_bench,...,gross_price,net_price,quantity,benchmark_count,hour,date,month,day_of_week_n,day_of_month,year
141580,BOG,2023-09-28,2023-09-28 13:57:00-05:00,Frutas & verduras,Tubérculos,-368218666129834277,Papa Sabanera Semi (Mediana) Kg,4000.6246,4000.0,4000.0,...,4030.0,4030.0,2.5,1.0,13,2023-09-28,9,3,28,2023
161232,BOG,2023-09-28,2023-09-28 14:42:00-05:00,Frutas & verduras,Tubérculos,-368218666129834277,Papa Sabanera Semi (Mediana) Kg,4000.6246,4000.0,4000.0,...,4030.0,4030.0,2.5,1.0,14,2023-09-28,9,3,28,2023
298830,BOG,2023-09-28,2023-09-28 15:45:00-05:00,Frutas & verduras,Tubérculos,-368218666129834277,Papa Sabanera Semi (Mediana) Kg,4000.6246,4000.0,4000.0,...,4460.0,4460.0,2.5,1.0,15,2023-09-28,9,3,28,2023
59533,BOG,2023-09-28,2023-09-28 16:33:00-05:00,Frutas & verduras,Tubérculos,-368218666129834277,Papa Sabanera Semi (Mediana) Kg,4000.6246,4000.0,4000.0,...,4030.0,4030.0,2.5,1.0,16,2023-09-28,9,3,28,2023
106616,BOG,2023-09-28,2023-09-28 16:33:00-05:00,Frutas & verduras,Tubérculos,-368218666129834277,Papa Sabanera Semi (Mediana) Kg,4000.6246,4000.0,4000.0,...,4030.0,4030.0,2.5,1.0,16,2023-09-28,9,3,28,2023
205065,BOG,2023-09-28,2023-09-28 16:20:00-05:00,Frutas & verduras,Tubérculos,-368218666129834277,Papa Sabanera Semi (Mediana) Kg,4000.6246,4000.0,4000.0,...,4030.0,4030.0,2.5,1.0,16,2023-09-28,9,3,28,2023
12852,BOG,2023-09-28,2023-09-28 17:44:00-05:00,Frutas & verduras,Tubérculos,-368218666129834277,Papa Sabanera Semi (Mediana) Kg,4000.6246,4000.0,4000.0,...,4030.0,4030.0,5.0,1.0,17,2023-09-28,9,3,28,2023
310833,BOG,2023-09-28,2023-09-28 17:59:00-05:00,Frutas & verduras,Tubérculos,-368218666129834277,Papa Sabanera Semi (Mediana) Kg,4000.6246,4000.0,4000.0,...,4030.0,4030.0,2.5,1.0,17,2023-09-28,9,3,28,2023
47683,BOG,2023-09-28,2023-09-28 18:47:00-05:00,Frutas & verduras,Tubérculos,-368218666129834277,Papa Sabanera Semi (Mediana) Kg,4000.6246,4000.0,4000.0,...,4030.0,4030.0,2.5,1.0,18,2023-09-28,9,3,28,2023
245402,BOG,2023-09-28,2023-09-28 18:36:00-05:00,Frutas & verduras,Tubérculos,-368218666129834277,Papa Sabanera Semi (Mediana) Kg,4000.6246,4000.0,4000.0,...,4030.0,4030.0,5.0,1.0,18,2023-09-28,9,3,28,2023


# Final

 # - Champiñón Blanco Paris 1Kg Bandeja  - 11117 rows
  # - Fresa Selecta (Grande) Kg  - 16084
  # - Papa Sabanera Gruesa (Grande) Kg - 2618 rows
  # - Papa Sabanera Semi (Mediana) Kg  - 3225 rows
  # - Rabano Estándar Kg   - mape 10558 rows

In [175]:
temp = get_filterd_dataset('Rabano Estándar Kg')
temp.sort_values('order_submited_date')

/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_30909/3257728416.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['order_submited_datetime'] = pd.to_datetime(filtered['order_submited_datetime'])


,region_code,order_submited_date,order_submited_datetime,parent_category,category,product_id,product_name,cost,benchmark,median_bench,avg_bench,max_bench,gross_price,net_price,quantity,benchmark_count
160712,BOG,2021-09-30,2021-09-30 00:34:00,Frutas & verduras,Verduras,353982459564085093,Rabano Estándar Kg,NaN,NaN,NaN,NaN,NaN,2070.0,2070.0,2.5,NaN
170412,BOG,2021-09-30,2021-09-30 03:02:00,Frutas & verduras,Verduras,353982459564085093,Rabano Estándar Kg,NaN,NaN,NaN,NaN,NaN,2070.0,2070.0,1.0,NaN
266840,BOG,2021-09-30,2021-09-30 18:07:00,Frutas & verduras,Verduras,353982459564085093,Rabano Estándar Kg,NaN,NaN,NaN,NaN,NaN,2070.0,2070.0,0.5,NaN
266837,BOG,2021-09-30,2021-09-30 15:59:00,Frutas & verduras,Verduras,353982459564085093,Rabano Estándar Kg,NaN,NaN,NaN,NaN,NaN,2070.0,2070.0,1.0,NaN
282479,BOG,2021-09-30,2021-09-30 23:30:00,Frutas & verduras,Verduras,353982459564085093,Rabano Estándar Kg,NaN,NaN,NaN,NaN,NaN,2070.0,2070.0,0.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39192,BOG,2023-09-29,2023-09-29 16:20:00,Frutas & verduras,Verduras,353982459564085093,Rabano Estándar Kg,4594.1957,4000.0,4000.0,4000.0,4000.0,3840.0,3840.0,1.0,1.0
151755,BOG,2023-09-29,2023-09-29 14:25:00,Frutas & verduras,Verduras,353982459564085093,Rabano Estándar Kg,4594.1957,4000.0,4000.0,4000.0,4000.0,3840.0,3840.0,0.5,1.0
144480,BOG,2023-09-29,2023-09-29 13:55:00,Frutas & verduras,Verduras,353982459564085093,Rabano Estándar Kg,4594.1957,4000.0,4000.0,4000.0,4000.0,3840.0,3840.0,0.5,1.0
18073,BOG,2023-09-29,2023-09-29 15:53:00,Frutas & verduras,Verduras,353982459564085093,Rabano Estándar Kg,4594.1957,4000.0,4000.0,4000.0,4000.0,3840.0,3840.0,0.5,1.0


In [290]:
from scipy import stats
import matplotlib.pyplot as plt
import pickle
from statsmodels.tsa.seasonal import seasonal_decompose
import pytz
from datetime import datetime

def get_filterd_dataset(product_name):

    new_df = pd.read_csv('data/created/Catalan_2023-10-17.csv')
    # new_df = new_df.dropna(how='all')

    filtered = new_df[new_df['product_name']==product_name]
    filtered['order_submited_datetime'] = pd.to_datetime(filtered['order_submited_datetime'])
    return filtered

def add_inflation(new_df):
    values = [
        3.68, 3.37, 3.14, 3.13, 3.16, 3.2, 3.12, 3.1, 3.23, 3.33, 3.27, 3.18, 3.15, 3.01, 3.21, 3.25, 3.31, 3.43, 3.79, 3.75,
        3.82, 3.86, 3.84, 3.8, 3.62, 3.72, 3.86, 3.51, 2.85, 2.19, 1.97, 1.88, 1.97, 1.75, 1.49, 1.61, 1.6, 1.56, 1.51, 1.95,
        3.3, 3.63, 3.97, 4.44, 4.51, 4.58, 5.26, 5.62, 6.94, 8.01, 8.53, 9.23, 9.07, 9.67, 10.21, 10.84, 11.44, 12.22, 12.53,
        13.12, 13.25, 13.28, 13.34, 12.82, 12.36, 12.13, 11.78, 11.43,10.79, 10.79
    ]

    # Create a list of dates from Jan 2018 to Aug 2023
    date_range = pd.date_range(start='2018-01-01', end='2023-10-30', freq='D')

    # Create a DataFrame

    inflation = []
    df = pd.DataFrame()
    df['date'] = date_range
    i = 0
    prev_month = df['date'][0].month
    # Populate the 'Price' column with values based on the corresponding month's value
    for date in df['date']:
        month = date.month
        if prev_month != month:
            prev_month = month
            i+=1
        inflation.append(values[i])
        # df.at[date, 'Price'] = value

    # Print the resulting DataFrame
    df['inflation'] = inflation
    # print(df)
    # df = df.set_index('date')
    df['date'] = pd.to_datetime(df['date'])
    new_df['date'] = pd.to_datetime(new_df['date'])
    # filtered2 = pd.merge(filtered2, df, how='left', left_index=True, right_index=True)
    merged = pd.merge(new_df, df, on='date', how='left')
    return merged

def generate_datafrane_for_profit2(date, price):
    test_date = pd.to_datetime(date)
    dates = [test_date.date()] * 24
    prices = [price] * 24
    hours = list(range(24))
    
 
    # Create DataFrame
    df = pd.DataFrame({'datetime': dates, 'hour': hours, 'net_price': prices})
    df['datetime'] = pd.to_datetime(df['datetime'])
    df['date'] = df['datetime'].dt.date
    df['day_of_week_n'] = df['datetime'].dt.day_of_week

    for day_num in range(0, 7):
        day_col_name = f'net_price_day_{day_num}'
        df[day_col_name] = df.apply(lambda row: row['net_price'] if row['day_of_week_n'] == day_num else 0, axis=1)
    # merged_Df = add_inflation(df)[columns[:-1]]
    df['date'] = df['date'].astype(str)
    merged_Df = df
    
    
    return merged_Df.sort_values(['date','hour'])

def mode(x):
    return stats.mode(x)[0][0] 

def add_to_Historic_Dataframe(new_data,date):
  # new_data = pd.read_csv('data/Frubana_2023_10_09.csv')
  
  historical =  pd.read_csv('historic_pilot_info_catalan.csv')
  colombia_tz = pytz.timezone('America/Bogota')
    # Convert the 'order_submited_datetime' column to Colombia timezone
  historical['order_submited_datetime'] = pd.to_datetime(historical['order_submited_datetime'])
  historical['order_submited_datetime'] = historical['order_submited_datetime'].dt.tz_convert(colombia_tz)

  new_data['order_submited_datetime'] = pd.to_datetime(new_data['order_submited_datetime'])
  new_data['order_submited_datetime'] = new_data['order_submited_datetime'].dt.tz_localize(pytz.utc).dt.tz_convert(colombia_tz)
  new_data['order_submited_date'] = new_data['order_submited_datetime'].dt.date.astype(str)
  
  new_data = new_data[new_data['order_submited_date']>date]

  common_columns = new_data.columns.intersection(historical.columns)
  merged_df = pd.concat([historical[common_columns], new_data[common_columns]])

  merged_df.sort_values('order_submited_datetime').to_csv(f'data/created/Catalan_{date}.csv')

  return merged_df.sort_values('order_submited_datetime')


def get_papa_grande():
    df = get_filterd_dataset('Papa Criolla Lavada Semi (Mediana) Kg')
    colombia_tz = pytz.timezone('America/Bogota')

    # Convert the 'order_submited_datetime' column to Colombia timezone
    if df['order_submited_datetime'].dt.tz is None:
    # If the datetime is naive, localize it
        df['order_submited_datetime'] = df['order_submited_datetime'].dt.tz_localize(pytz.utc).dt.tz_convert(colombia_tz)
    else:
        # If the datetime is already timezone-aware, just convert it
        df['order_submited_datetime'] = df['order_submited_datetime'].dt.tz_convert(colombia_tz)

    df['order_submited_datetime'] = pd.to_datetime(df['order_submited_datetime'])
    df['hour'] = df['order_submited_datetime'].dt.hour
    df['date'] = df['order_submited_datetime'].dt.date
    df['month'] = df['order_submited_datetime'].dt.month
    # df['day_of_week_n'] = df['order_submited_datetime'].dt.day_of_week
    df['day_of_month'] = df['order_submited_datetime'].dt.day
    df['year'] = df['order_submited_datetime'].dt.year
    agg_df = df.groupby(['date', 'hour']).agg({
    'net_price': mode,
    'quantity': 'sum',
    'cost': 'first',
    'avg_bench': 'first',
    'product_id': 'first',
    'product_name': 'first',
    'gross_price': 'first',
    # 'day_of_week_n': 'first',
    'day_of_month': 'first',
    # 'date': 'first'
    'year':'first',
    'month': 'first'

    }).reset_index()

    min_date = pd.to_datetime(agg_df['date']).min()
    max_date = pd.to_datetime(agg_df['date']).max()

    date_range = pd.date_range(start=min_date, end=max_date, freq='D')
    hour_range = list(range(24))

    full_df = pd.DataFrame([(date, hour) for date in date_range for hour in hour_range], columns=['date', 'hour'])
    full_df['date'] = full_df['date'].dt.date.astype(str)  # Convert datetime to string to match df's 'date' format
    temp = agg_df[['date','hour']]
    temp_unique = temp.drop_duplicates()
    temp_unique['date'] = pd.to_datetime(temp_unique['date'])
    temp_unique['date'] = temp_unique['date'].dt.date.astype(str)
    # Identify missing rows
    merged_df = pd.merge(full_df, temp_unique, on=['date', 'hour'], how='left', indicator=True)
    missing_rows = merged_df[merged_df['_merge'] == 'left_only'][['date', 'hour']]
    # missing_rows.head(23)
    agg_df['date'] = pd.to_datetime(agg_df['date'])
    agg_df['date'] = agg_df['date'].dt.date.astype(str)
    merged_df.to_csv('temp.csv')
    
    merged_df = pd.merge(agg_df, missing_rows, on=['date', 'hour'], how='outer')
    merged_df.to_csv('temp3.csv')
    
    new_date = merged_df.iloc[-1]['date']
    new_price = merged_df.iloc[-1]['net_price']


    date_format = '%Y-%m-%d'
    date_DATE = datetime.strptime(new_date, date_format)
    # Add one day to the date
    next_date = date_DATE + timedelta(days=1)
    next_date_str = next_date.strftime(date_format)
    print('next date',next_date_str)
    test_vector_copy = generate_datafrane_for_profit2(next_date_str,new_price )
    print('new_test_vecotr',test_vector_copy.iloc[-1]['date'])
    merged_df = merged_df.sort_values(['date','hour'])
    merged_df = pd.concat([ merged_df, test_vector_copy])
    merged_df.to_csv('temp4.csv')
    
    print('new_row',merged_df.iloc[-1]['date'])


    cols_to_fill = [col for col in merged_df.columns if col not in ['quantity']]
    merged_df = merged_df.sort_values(['date','hour'])
    
    merged_df['net_price'] = merged_df['net_price'].fillna(method='ffill')
    merged_df['quantity'].fillna(0, inplace=True)
    merged_df.to_csv('temp5.csv')
    
    merged_df['net_price_FP'] = merged_df['net_price']
    merged_df['quantity_FP'] = merged_df['quantity']
    merged_df['net_price_FP_lag48'] = merged_df['net_price_FP'].shift(48)
    merged_df['net_price_FP_lag49'] = merged_df['net_price_FP'].shift(49)
    merged_df['quantity_FP_lag48'] = merged_df['quantity_FP'].shift(48)
    merged_df['quantity_FP_lag49'] = merged_df['quantity_FP'].shift(49)
    
    # if not comp
    daily_quantity_papagrande = merged_df.groupby('date').agg({'quantity': 'sum','net_price': mode}).reset_index()
    daily_quantity_papagrande.to_csv('temp1.csv')
    
    daily_quantity_papagrande['FP_rolling_avg_quantity_7'] = (
        daily_quantity_papagrande['quantity']
        .rolling(window=7)
        .mean()
        .shift(2)  # This adds the 1-day lag
    )

    daily_quantity_papagrande['FP_rolling_avg_quantity_30'] = (
        daily_quantity_papagrande['quantity']
        .rolling(window=30)
        .mean()
        .shift(2)  # This adds the 1-day lag
    )

    daily_quantity_papagrande['FP_quantity_daily_lag2'] = (
            daily_quantity_papagrande['quantity']
            .shift(2)  # This adds the 1-day lag
        )
    daily_quantity_papagrande['FP_net_price_daily_lag2'] = (
            daily_quantity_papagrande['net_price']
            .shift(2)  # This adds the 1-day lag
        )

    daily_quantity_papagrande['FP_net_price_daily_lag1'] = (
            daily_quantity_papagrande['net_price']
            .shift(1)  # This adds the 1-day lag
        )

    daily_quantity_papagrande['FP_daily_net_price'] = daily_quantity_papagrande['net_price']
    daily_quantity_papagrande['FP_daily_quantity'] = daily_quantity_papagrande['quantity']
    daily_quantity_papagrande['date']= daily_quantity_papagrande['date'].astype(str)
    merged_df['date']= merged_df['date'].astype(str)
    
    return daily_quantity_papagrande, merged_df



def aggregate_by_hour(df, time_bucket, has_competitor_price):
    """
    Aggregate a DataFrame by a specified time bucket (e.g., 24 for daily aggregation).
    
    Args:
        df (pd.DataFrame): Input DataFrame with a datetime field.
        time_bucket (int): Number of hours in each time bucket for aggregation.
    
    Returns:
        pd.DataFrame: Aggregated DataFrame with the datetime bucketed and values aggregated.
    """
    # Create a timezone object for Colombia (CST)
    colombia_tz = pytz.timezone('America/Bogota')
    
    # Convert the 'order_submited_datetime' column to Colombia timezone
    df['order_submited_datetime'] = df['order_submited_datetime'].dt.tz_convert(colombia_tz)

    df['order_submited_datetime'] = pd.to_datetime(df['order_submited_datetime'])
    df['hour'] = df['order_submited_datetime'].dt.hour
    df['date'] = df['order_submited_datetime'].dt.date
    df['month'] = df['order_submited_datetime'].dt.month
    # df['day_of_week_n'] = df['order_submited_datetime'].dt.day_of_week
    df['day_of_month'] = df['order_submited_datetime'].dt.day
    df['year'] = df['order_submited_datetime'].dt.year
    agg_df = df.groupby(['date', 'hour', 'net_price']).agg({
    'quantity': 'sum',
    'cost': 'first',
    'avg_bench': 'first',
    'product_id': 'first',
    'product_name': 'first',
    'gross_price': 'first',
    # 'day_of_week_n': 'first',
    'day_of_month': 'first',
    # 'date': 'first'
    'year':'first',
    'month': 'first'

}).reset_index()

    min_date = pd.to_datetime(agg_df['date']).min()
    max_date = pd.to_datetime(agg_df['date']).max()

    date_range = pd.date_range(start=min_date, end=max_date, freq='D')
    hour_range = list(range(24))

    full_df = pd.DataFrame([(date, hour) for date in date_range for hour in hour_range], columns=['date', 'hour'])
    full_df['date'] = full_df['date'].dt.date.astype(str)  # Convert datetime to string to match df's 'date' format
    temp = agg_df[['date','hour']]
    temp_unique = temp.drop_duplicates()
    temp_unique['date'] = pd.to_datetime(temp_unique['date'])
    temp_unique['date'] = temp_unique['date'].dt.date.astype(str)
    # Identify missing rows
    merged_df = pd.merge(full_df, temp_unique, on=['date', 'hour'], how='left', indicator=True)
    missing_rows = merged_df[merged_df['_merge'] == 'left_only'][['date', 'hour']]
    # missing_rows.head(23)
    agg_df['date'] = pd.to_datetime(agg_df['date'])
    agg_df['date'] = agg_df['date'].dt.date.astype(str)
    merged_df = pd.merge(agg_df, missing_rows, on=['date', 'hour'], how='outer')
    new_date = merged_df.iloc[-1]['date']
    new_price = merged_df.iloc[-1]['net_price']


    date_format = '%Y-%m-%d'
    date_DATE = datetime.strptime(new_date, date_format)
    # Add one day to the date
    next_date = date_DATE + timedelta(days=1)
    next_date_str = next_date.strftime(date_format)
    print('next date',next_date_str)
    test_vector_copy = generate_datafrane_for_profit2(next_date_str,new_price )
    print('new_test_vecotr',test_vector_copy.iloc[-1]['date'])
    merged_df = merged_df.sort_values(['date','hour'])
    merged_df = pd.concat([ merged_df, test_vector_copy])
    print('new_row',merged_df.iloc[-1]['date'])

    # test_vector_copy = generate_datafrane_for_profit2('2023-10-18',new_price )
    # print('new_test_vecotr',test_vector_copy.iloc[-1]['date'])
    # merged_df = merged_df.sort_values(['date','hour'])
    # merged_df = pd.concat([ merged_df, test_vector_copy])
    # print('new_row',merged_df.iloc[-1]['date'])


    cols_to_fill = [col for col in merged_df.columns if col not in ['quantity']]
    merged_df = merged_df.sort_values(['date','hour'])
    merged_df['net_price'] = merged_df['net_price'].fillna(method='ffill')
   
    
    
        
    merged_df['quantity'].fillna(0, inplace=True)

    # if not comp
    daily_quantity = merged_df.groupby('date').agg({'quantity': 'sum', 'net_price': 'mean'}).reset_index()


    result = seasonal_decompose(daily_quantity['quantity'], model='additive', period=7)
    daily_quantity['residual_daily'] = result.resid
    daily_quantity['trend_daily'] = result.trend
    daily_quantity['seasonal_daily'] = result.seasonal
    daily_quantity['residual_daily_lag2'] = daily_quantity['residual_daily'].shift(2)
    daily_quantity['trend_daily_lag2'] = daily_quantity['trend_daily'].shift(2)
    daily_quantity['seasonal_daily_lag2'] = daily_quantity['seasonal_daily'].shift(2)
   
    #  if  comp
    # daily_quantity = merged_df.groupby('date').agg({'quantity': 'sum', 'net_price': 'mean','avg_bench':'mean'}).reset_index()

    # Step 2: Calculate 7-Day Rolling Average with 1-day Lag
    daily_quantity['rolling_avg_quantity_7_lag2'] = (
        daily_quantity['quantity']
        .rolling(window=7)
        .mean()
        .shift(2)  # This adds the 1-day lag
    )

    daily_quantity['rolling_avg_quantity_10_lag2'] = (
        daily_quantity['quantity']
        .rolling(window=10)
        .mean()
        .shift(2)  # This adds the 1-day lag
    )

    daily_quantity['rolling_avg_quantity_15_lag2'] = (
        daily_quantity['quantity']
        .rolling(window=15)
        .mean()
        .shift(2)  # This adds the 1-day lag
    )

    daily_quantity['rolling_avg_quantity_30_lag2'] = (
        daily_quantity['quantity']
        .rolling(window=30)
        .mean()
        .shift(2)  # This adds the 1-day lag
    )

    daily_quantity['quantity_daily_lag2'] = (
        daily_quantity['quantity']
        .shift(2)  # This adds the 1-day lag
    )

    daily_quantity['ratio_rolling_avg_quantity_30_7_lag2'] = daily_quantity['rolling_avg_quantity_30_lag2']/daily_quantity['rolling_avg_quantity_7_lag2']
    

    def ewm_features(dataframe, alphas, lags):
      for alpha in alphas:
          for lag in lags:
              dataframe['quantity_ewm_alpha_' + str(alpha).replace(".", "") + "_lag_" + str(lag)] = \
                  dataframe['quantity'].transform(lambda x: x.shift(lag).ewm(alpha=alpha).mean())
      dataframe.fillna(0, inplace=True)
      return dataframe
    
    alphas = [0.95, 0.5]
    lags   = [2,7,10,14,30,60]


    daily_quantity= ewm_features(daily_quantity, alphas, lags)


    # daily_quantity['diff'] = daily_quantity['net_price'] - daily_quantity['avg_bench']
    # daily_quantity['avg_bench_lag1'] = daily_quantity['avg_bench'].shift(1)
    # daily_quantity['diff_cross'] = daily_quantity['net_price'] - daily_quantity['avg_bench_lag1']

    # daily_quantity['diff_lag2'] = (
    #     daily_quantity['diff']
    #     .shift(2)
    #  # This adds the 1-day lag
    # )
    # daily_quantity['diff_cross_lag2'] = (
    #     daily_quantity['diff_cross']
    #     .shift(2)
    #  # This adds the 1-day lag
    # )

    # merged_df = merged_df.merge(daily_quantity[['date', 'rolling_avg_quantity_30','rolling_avg_quantity_7','quantity_daily_lag2']], on='date', how='left')
    
    merged_df = merged_df.merge(daily_quantity[['date', 'ratio_rolling_avg_quantity_30_7_lag2','quantity_ewm_alpha_095_lag_2', 'quantity_ewm_alpha_095_lag_7','quantity_ewm_alpha_095_lag_10','quantity_ewm_alpha_095_lag_14'
                                                ,'quantity_ewm_alpha_095_lag_30','quantity_ewm_alpha_095_lag_60','quantity_ewm_alpha_05_lag_2', 'quantity_ewm_alpha_05_lag_7','quantity_ewm_alpha_05_lag_10','quantity_ewm_alpha_05_lag_14'
                                                ,'quantity_ewm_alpha_05_lag_30','quantity_ewm_alpha_05_lag_60',
                                                'rolling_avg_quantity_30_lag2','rolling_avg_quantity_15_lag2','rolling_avg_quantity_10_lag2','rolling_avg_quantity_7_lag2','quantity_daily_lag2','trend_daily_lag2','seasonal_daily_lag2','residual_daily_lag2']], on='date', how='left')
    
    daily_quantity_FP, hourly_FP = get_papa_grande()
    merged_df = merged_df.merge(daily_quantity_FP[['date','FP_net_price_daily_lag1','FP_net_price_daily_lag2', 'FP_quantity_daily_lag2','FP_rolling_avg_quantity_30','FP_rolling_avg_quantity_7','FP_daily_quantity','FP_daily_net_price']], on='date', how='left')

    merged_df = merged_df.merge(hourly_FP[['date','hour','net_price_FP_lag48','quantity_FP_lag48','net_price_FP_lag49','quantity_FP_lag49','net_price_FP','quantity_FP']], on=['date', 'hour'], how='left')

    merged_df = merged_df.sort_values(['date','hour'])
    merged_df['date']=merged_df['date'].astype(str)
    
    merged_df = merged_df.sort_values(['date','hour'])
    
    return merged_df



def create_features(dataframe, has_competitor_price):
    def is_weekend(day):
        return 1 if day in [5, 6] else 0
    
    # dataframe['datetime'] = pd.to_datetime(dataframe['date'])
    
    cols_to_fill = [col for col in dataframe.columns if col not in ['quantity']]
    dataframe = dataframe.sort_values(['date','hour'])
    # dataframe['net_price'] = dataframe['net_price'].fillna(method='ffill')
    dataframe['quantity'].fillna(0, inplace=True)

    dataframe = add_inflation(dataframe)
    dataframe['day_of_week_n'] = dataframe['date'].dt.day_of_week

    dataframe['if_weekend'] = dataframe['day_of_week_n'].apply(is_weekend)
    for day_num in range(0, 7):
        day_col_name = f'net_price_day_{day_num}'
        dataframe[day_col_name] = dataframe.apply(lambda row: row['net_price'] if row['day_of_week_n'] == day_num else 0, axis=1)

    # dataframe.set_index('bucketed_datetime', inplace=True)
    # dataframe.index.freq = 'D' 
    # dataframe = dataframe.dropna()
    print('features ', dataframe.info())
    return dataframe




In [292]:
time_bucket=24
has_competitor_price=1


filtered1 = get_filterd_dataset('Papa Criolla Segunda Mixta Kg')
# filterd2 = get_filterd_dataset('Papa Sabanera Gruesa (Grande) Kg')

daily_aggregated_df = aggregate_by_hour(filtered1, time_bucket=time_bucket, has_competitor_price=has_competitor_price)
features = create_features(daily_aggregated_df, has_competitor_price)
# features[features['date']=='2021-09-29']
# daily_aggregated_df[daily_aggregated_df['date']=='2023-09-28']
features = features.sort_values(['date','hour'])
# featuresPapa Sabanera Gruesa (Grande) Kg

/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_14780/331819806.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['order_submited_datetime'] = pd.to_datetime(filtered['order_submited_datetime'])
/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_14780/331819806.py:284: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_unique['date'] = pd.to_datetime(temp_unique['date'])
/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_14780/331819806.py:285: SettingWithCopyWarning: 

next date 2023-10-18
new_test_vecotr 2023-10-18
new_row 2023-10-18


/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_14780/331819806.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['order_submited_datetime'] = pd.to_datetime(filtered['order_submited_datetime'])
/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_14780/331819806.py:78: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  return stats.mode(x)[0][0]


next date 2023-10-18
new_test_vecotr 2023-10-18
new_row 2023-10-18


/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_14780/331819806.py:78: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  return stats.mode(x)[0][0]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18197 entries, 0 to 18196
Data columns (total 57 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   date                                  18197 non-null  datetime64[ns]
 1   hour                                  18197 non-null  int64         
 2   net_price                             18176 non-null  float64       
 3   quantity                              18197 non-null  float64       
 4   cost                                  4411 non-null   float64       
 5   avg_bench                             323 non-null    float64       
 6   product_id                            5057 non-null   float64       
 7   product_name                          5057 non-null   object        
 8   gross_price                           5057 non-null   float64       
 9   day_of_month                          5057 non-null   float64       
 10

In [294]:
features

,date,hour,net_price,quantity,cost,avg_bench,product_id,product_name,gross_price,day_of_month,...,FP_daily_quantity,FP_daily_net_price,net_price_FP_lag48,quantity_FP_lag48,net_price_FP_lag49,quantity_FP_lag49,net_price_FP,quantity_FP,inflation,if_weekend
0,2021-09-29,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,21.5,NaN,NaN,NaN,NaN,NaN,NaN,0.0,4.51,0
1,2021-09-29,1,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,21.5,NaN,NaN,NaN,NaN,NaN,NaN,0.0,4.51,0
2,2021-09-29,2,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,21.5,NaN,NaN,NaN,NaN,NaN,NaN,0.0,4.51,0
3,2021-09-29,3,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,21.5,NaN,NaN,NaN,NaN,NaN,NaN,0.0,4.51,0
4,2021-09-29,4,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,21.5,NaN,NaN,NaN,NaN,NaN,NaN,0.0,4.51,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18192,2023-10-18,19,2675.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,3395.77,3880.00,38.0,3880.00,37.0,3395.77,0.0,10.79,0
18193,2023-10-18,20,2675.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,3395.77,3770.00,14.0,3880.00,38.0,3395.77,0.0,10.79,0
18194,2023-10-18,21,2675.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,3395.77,3880.00,21.5,3770.00,14.0,3395.77,0.0,10.79,0
18195,2023-10-18,22,2675.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,3395.77,3668.21,11.0,3880.00,21.5,3395.77,0.0,10.79,0


In [277]:
features[[
    'date', 'net_price', 'day_of_week_n', 'net_price_day_0', 'net_price_day_1',
    'net_price_day_2', 'net_price_day_3', 'net_price_day_4', 'net_price_day_5',
    'net_price_day_6', 'hour', 'quantity_ewm_alpha_095_lag_7',
    'FP_quantity_daily_lag2', 'trend_daily_lag2', 'quantity'
]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18316 entries, 0 to 18315
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   date                          18316 non-null  datetime64[ns]
 1   net_price                     18295 non-null  float64       
 2   day_of_week_n                 18316 non-null  int32         
 3   net_price_day_0               18316 non-null  float64       
 4   net_price_day_1               18316 non-null  float64       
 5   net_price_day_2               18295 non-null  float64       
 6   net_price_day_3               18316 non-null  float64       
 7   net_price_day_4               18316 non-null  float64       
 8   net_price_day_5               18316 non-null  float64       
 9   net_price_day_6               18316 non-null  float64       
 10  hour                          18316 non-null  int64         
 11  quantity_ewm_alpha_095_lag_7

In [264]:

# features[['date','day_of_week_n']].head(30)
features['day_of_week_n'] = features['date'].dt.day_of_week
features
features[['date','inflation']].tail(30)

,date,inflation
18009,2023-10-17,10.79
18010,2023-10-17,10.79
18011,2023-10-17,10.79
18012,2023-10-17,10.79
18013,2023-10-17,10.79
18014,2023-10-17,10.79
18015,2023-10-18,10.79
18016,2023-10-18,10.79
18017,2023-10-18,10.79
18018,2023-10-18,10.79


In [168]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18063 entries, 0 to 18062
Data columns (total 57 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   date                                  18063 non-null  datetime64[ns]
 1   hour                                  18063 non-null  int64         
 2   net_price                             18044 non-null  float64       
 3   quantity                              18063 non-null  float64       
 4   cost                                  2537 non-null   float64       
 5   avg_bench                             386 non-null    float64       
 6   product_id                            2864 non-null   float64       
 7   product_name                          2864 non-null   object        
 8   gross_price                           2864 non-null   float64       
 9   day_of_month                          2864 non-null   float64       
 10

In [179]:
df = pd.read_csv('combined_data/PapaCriollaSM_LS(mode).csv')
df[[
    'date', 'net_price', 'day_of_week_n', 'net_price_day_0', 'net_price_day_1',
    'net_price_day_2', 'net_price_day_3', 'net_price_day_4', 'net_price_day_5',
    'net_price_day_6', 'hour', 'quantity_ewm_alpha_095_lag_7',
    'FP_quantity_daily_lag2', 'trend_daily_lag2', 'quantity'
]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18015 entries, 0 to 18014
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   date                          18015 non-null  object 
 1   net_price                     17996 non-null  float64
 2   day_of_week_n                 18015 non-null  int64  
 3   net_price_day_0               18015 non-null  float64
 4   net_price_day_1               18015 non-null  float64
 5   net_price_day_2               17996 non-null  float64
 6   net_price_day_3               18015 non-null  float64
 7   net_price_day_4               18015 non-null  float64
 8   net_price_day_5               18015 non-null  float64
 9   net_price_day_6               18015 non-null  float64
 10  hour                          18015 non-null  int64  
 11  quantity_ewm_alpha_095_lag_7  18015 non-null  float64
 12  FP_quantity_daily_lag2        17943 non-null  float64
 13  t

/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_14780/3301687076.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('combined_data/PapaSabaneraGG_SM(mode).csv')


In [295]:
features.to_csv('combined_data/PapaCriollaSM_LS(mode).csv')

In [167]:
data = pd.read_csv('temp1.csv')
data[data['net_price'].isna()]

,Unnamed: 0,date,quantity,net_price
0,0,2021-09-29,17.5,NaN


In [262]:
time_bucket=24
has_competitor_price=1


filtered1 = get_filterd_dataset('Champiñón Blanco Paris 500 gr Bandeja')

filtered1

/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_30909/3257728416.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['order_submited_datetime'] = pd.to_datetime(filtered['order_submited_datetime'])


,region_code,order_submited_date,order_submited_datetime,parent_category,category,product_id,product_name,cost,benchmark,median_bench,avg_bench,max_bench,gross_price,net_price,quantity,benchmark_count
16,BOG,2022-10-21,2022-10-21 18:00:00,Frutas & verduras,Verduras,-1073190491589771159,Champiñón Blanco Paris 500 gr Bandeja,11054.7233,NaN,NaN,NaN,NaN,14910.0,14462.7,1.0,NaN
40,BOG,2022-10-18,2022-10-18 19:43:00,Frutas & verduras,Verduras,-1073190491589771159,Champiñón Blanco Paris 500 gr Bandeja,11315.9954,NaN,NaN,NaN,NaN,14400.0,13968.0,1.0,NaN
62,BOG,2022-10-25,2022-10-25 01:36:00,Frutas & verduras,Verduras,-1073190491589771159,Champiñón Blanco Paris 500 gr Bandeja,10952.7040,NaN,NaN,NaN,NaN,14860.0,14414.2,1.0,NaN
67,BOG,2022-10-26,2022-10-26 21:06:00,Frutas & verduras,Verduras,-1073190491589771159,Champiñón Blanco Paris 500 gr Bandeja,10828.7743,NaN,NaN,NaN,NaN,15120.0,14666.4,1.0,NaN
74,BOG,2022-10-27,2022-10-27 01:41:00,Frutas & verduras,Verduras,-1073190491589771159,Champiñón Blanco Paris 500 gr Bandeja,10828.7743,NaN,NaN,NaN,NaN,15120.0,14666.4,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318685,BOG,2023-09-19,2023-09-19 18:44:00,Frutas & verduras,Verduras,-1073190491589771159,Champiñón Blanco Paris 500 gr Bandeja,12636.6499,13000.0,13000.0,13000.0,13000.0,12780.0,12780.0,1.0,1.0
318690,BOG,2023-09-20,2023-09-20 02:13:00,Frutas & verduras,Verduras,-1073190491589771159,Champiñón Blanco Paris 500 gr Bandeja,12646.4460,13000.0,13000.0,13000.0,13000.0,12780.0,12780.0,1.0,1.0
318751,BOG,2023-09-28,2023-09-28 00:31:00,Frutas & verduras,Verduras,-1073190491589771159,Champiñón Blanco Paris 500 gr Bandeja,12548.0995,12000.0,12000.0,12000.0,12000.0,13040.0,11996.8,1.0,1.0
318752,BOG,2023-09-28,2023-09-28 01:22:00,Frutas & verduras,Verduras,-1073190491589771159,Champiñón Blanco Paris 500 gr Bandeja,12548.0995,12000.0,12000.0,12000.0,12000.0,13040.0,11996.8,2.0,1.0


In [300]:
import pandas as pd
import numpy as np
def addDataframe(new_data,date):
  # new_data = pd.read_csv('data/Frubana_2023_10_09.csv')
  
  historical =  pd.read_csv('historic_pilot_info_catalan.csv')
  colombia_tz = pytz.timezone('America/Bogota')
    # Convert the 'order_submited_datetime' column to Colombia timezone
  historical['order_submited_datetime'] = pd.to_datetime(historical['order_submited_datetime'])
  historical['order_submited_datetime'] = historical['order_submited_datetime'].dt.tz_localize(pytz.utc).dt.tz_convert(colombia_tz)

  new_data['order_submited_datetime'] = pd.to_datetime(new_data['order_submited_datetime'])
  new_data['order_submited_datetime'] = new_data['order_submited_datetime'].dt.tz_localize(pytz.utc).dt.tz_convert(colombia_tz)
  new_data['order_submited_date'] = new_data['order_submited_datetime'].dt.date.astype(str)
  
  new_data = new_data[new_data['order_submited_date']>date]

  common_columns = new_data.columns.intersection(historical.columns)
  merged_df = pd.concat([historical[common_columns], new_data[common_columns]])


  return merged_df.sort_values('order_submited_datetime')

data = pd.read_csv('data/Frubana_2023_10_09.csv')
addDataframe(data,'2023-09-29')


,region_code,order_submited_date,order_submited_datetime,parent_category,category,product_id,product_name,cost,benchmark,median_bench,avg_bench,max_bench,gross_price,net_price,quantity,benchmark_count
70286,BOG,2021-09-30,2021-09-29 19:06:00-05:00,Frutas & verduras,Verduras,-6536998881607504388,Apio Estándar (Grande) Atado,NaN,2000.0,2000.0,2000.0,2000.0,2316.00,1968.6,1.0,1.0
101821,BOG,2021-09-30,2021-09-29 19:06:00-05:00,Frutas & verduras,Verduras,-882794291467029550,Pimentón Maduración Mixta Semi (Mediano / Nata...,NaN,2800.0,2800.0,2800.0,2800.0,3028.00,2573.8,1.5,1.0
39193,BOG,2021-09-30,2021-09-29 19:09:00-05:00,Frutas & verduras,Verduras,-1160551012065768576,Champiñón Blanco Paris 1Kg Bandeja,NaN,16000.0,16000.0,16000.0,16000.0,20470.00,20470.0,2.0,1.0
282474,BOG,2021-09-30,2021-09-29 19:09:00-05:00,Frutas & verduras,Frutas,1260412819932663981,Fresa Selecta (Grande) Kg,NaN,NaN,NaN,NaN,NaN,8219.00,8219.0,1.0,NaN
35023,BOG,2021-09-30,2021-09-29 19:09:00-05:00,Frutas & verduras,Verduras,3038455164096626586,Guatila Estándar Kg,NaN,NaN,NaN,NaN,NaN,1570.00,1570.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38742,BOG,2023-10-09,2023-10-09 09:32:00-05:00,Frutas & verduras,Tubérculos,7888420588372063556,Papa Criolla Sin Lavar Tamaño Mixto Kg,1899.3450,2667.0,2667.0,2667.0,2667.0,2260.00,2260.0,4.0,1.0
68303,BOG,2023-10-09,2023-10-09 09:33:00-05:00,Frutas & verduras,Verduras,-6536998881607504388,Apio Estándar (Grande) Atado,615.0477,1333.0,1333.0,1333.0,1333.0,1540.00,1540.0,2.0,1.0
117924,BOG,2023-10-09,2023-10-09 09:33:00-05:00,Frutas & verduras,Verduras,-882794291467029550,Pimentón Maduración Mixta Semi (Mediano / Nata...,1821.8443,3500.0,3500.0,3500.0,3500.0,3360.00,3360.0,7.0,1.0
271578,BOG,2023-10-09,2023-10-09 09:34:00-05:00,Frutas & verduras,Verduras,-882794291467029550,Pimentón Maduración Mixta Semi (Mediano / Nata...,1821.8443,3500.0,3500.0,3500.0,3500.0,3630.00,3630.0,2.0,1.0


<!-- 318760 rows × 16 columns historic -->
<!-- 5865 rows × 17 columns only new -->

In [304]:
data = pd.read_csv('data/Frubana_2023_10_09.csv')
data = data[data['order_submited_date']>'2023-09-29']
data

,region_code,order_submited_date,order_submited_datetime,parent_category,category,product_id,product_name,cost,benchmark,median_bench,avg_bench,max_bench,gross_price,net_price,quantity,product_step_unit,benchmark_count
18910,BOG,2023-09-30,2023-09-30 13:26,Frutas & verduras,Verduras,7749572820280257375,Cebolla Cabezona Roja Sin Pelar Tamaño Mixto D...,4345.9175,3750.0,3750.0,3750.0,3750.0,3600.84,3410.00,10.0,5.0,1.0
18911,BOG,2023-09-30,2023-09-30 17:22,Frutas & verduras,Tubérculos,-3386165639547488916,Papa Criolla Lavada Semi (Mediana) Kg,3069.9237,3571.0,3571.0,3571.0,3571.0,3450.00,3450.00,5.0,2.5,1.0
18912,BOG,2023-09-30,2023-09-30 21:12,Frutas & verduras,Verduras,-6536998881607504388,Apio Estándar (Grande) Atado,615.0477,1333.0,1333.0,1333.0,1333.0,1380.00,1342.74,1.0,1.0,1.0
18913,BOG,2023-10-01,2023-10-01 16:09,Frutas & verduras,Verduras,-6536998881607504388,Apio Estándar (Grande) Atado,NaN,1333.0,1333.0,1333.0,1333.0,1260.00,1225.98,3.0,3.0,1.0
18914,BOG,2023-10-01,2023-10-01 22:59,Frutas & verduras,Verduras,353982459564085093,Rabano Estándar Kg,NaN,4000.0,4000.0,4000.0,4000.0,3840.00,3840.00,0.5,0.5,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322765,BOG,2023-10-08,2023-10-08 22:21,Frutas & verduras,Verduras,-6536998881607504388,Apio Estándar (Grande) Atado,597.9026,1333.0,1333.0,1333.0,1333.0,1540.00,1540.00,1.0,1.0,1.0
322766,BOG,2023-10-08,2023-10-08 23:25,Frutas & verduras,Verduras,-6536998881607504388,Apio Estándar (Grande) Atado,597.9026,1333.0,1333.0,1333.0,1333.0,1410.00,1371.93,9.0,3.0,1.0
322767,BOG,2023-10-09,2023-10-09 00:25,Frutas & verduras,Verduras,353982459564085093,Rabano Estándar Kg,3064.8017,NaN,NaN,NaN,NaN,3840.00,3840.00,2.0,0.5,NaN
322768,BOG,2023-10-09,2023-10-09 01:02,Frutas & verduras,Tubérculos,7888420588372063556,Papa Criolla Sin Lavar Tamaño Mixto Kg,1899.3450,2667.0,2667.0,2667.0,2667.0,2260.00,2260.00,4.0,4.0,1.0


In [306]:
historical =  pd.read_csv('historic_pilot_info_catalan.csv')
colombia_tz = pytz.timezone('America/Bogota')
    # Convert the 'order_submited_datetime' column to Colombia timezone
historical['order_submited_datetime'] = pd.to_datetime(historical['order_submited_datetime'])
historical['order_submited_datetime'] = historical['order_submited_datetime'].dt.tz_localize(pytz.utc).dt.tz_convert(colombia_tz)
historical

,region_code,order_submited_date,order_submited_datetime,parent_category,category,product_id,product_name,cost,benchmark,median_bench,avg_bench,max_bench,gross_price,net_price,quantity,benchmark_count
0,BOG,2022-10-18,2022-10-18 15:47:00-05:00,Frutas & verduras,Verduras,-6536998881607504388,Apio Estándar (Grande) Atado,901.3589,1000.0,1100.0,1100.0,1200.0,1440.0,1440.00,1.0,2.0
1,BOG,2022-10-19,2022-10-18 20:15:00-05:00,Frutas & verduras,Verduras,7749572820280257375,Cebolla Cabezona Roja Sin Pelar Tamaño Mixto D...,2899.9498,2916.0,2916.0,2916.0,2916.0,3605.0,3352.65,2.0,1.0
2,BOG,2022-10-19,2022-10-18 20:06:00-05:00,Frutas & verduras,Tubérculos,-8910485076695139605,Papa Criolla Segunda Mixta Kg,2089.4632,NaN,NaN,NaN,NaN,2660.0,2660.00,10.0,NaN
3,BOG,2022-10-20,2022-10-20 09:49:00-05:00,Frutas & verduras,Verduras,-882794291467029550,Pimentón Maduración Mixta Semi (Mediano / Nata...,1876.7926,2500.0,2500.0,2500.0,2500.0,2450.0,2450.00,10.0,2.0
4,BOG,2022-10-19,2022-10-19 14:01:00-05:00,Frutas & verduras,Verduras,5533752035170569793,Cebolla Cabezona Roja Limpia Tamaño Mixto Kg (...,3985.4596,NaN,NaN,NaN,NaN,8112.0,8112.00,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318755,BOG,2023-09-28,2023-09-28 17:36:00-05:00,Frutas & verduras,Tubérculos,7888420588372063556,Papa Criolla Sin Lavar Tamaño Mixto Kg,1941.9497,2222.0,2261.0,2261.0,2300.0,2400.0,2400.00,4.0,2.0
318756,BOG,2023-09-29,2023-09-28 21:55:00-05:00,Frutas & verduras,Verduras,353982459564085093,Rabano Estándar Kg,4594.1957,4000.0,4000.0,4000.0,4000.0,3840.0,3840.00,1.0,1.0
318757,BOG,2023-09-29,2023-09-29 09:51:00-05:00,Frutas & verduras,Tubérculos,-3386165639547488916,Papa Criolla Lavada Semi (Mediana) Kg,3723.2401,3372.0,3372.0,3372.0,3372.0,3260.0,3260.00,2.5,1.0
318758,BOG,2023-09-21,2023-09-20 22:19:00-05:00,Frutas & verduras,Verduras,-1073190491589771159,Champiñón Blanco Paris 500 gr Bandeja,12616.8519,12500.0,12750.0,12750.0,13000.0,12790.0,11766.80,1.0,2.0


In [307]:
historical =  pd.read_csv('historic_pilot_info_catalan.csv')
historical

,region_code,order_submited_date,order_submited_datetime,parent_category,category,product_id,product_name,cost,benchmark,median_bench,avg_bench,max_bench,gross_price,net_price,quantity,benchmark_count
0,BOG,2022-10-18,2022-10-18 20:47,Frutas & verduras,Verduras,-6536998881607504388,Apio Estándar (Grande) Atado,901.3589,1000.0,1100.0,1100.0,1200.0,1440.0,1440.00,1.0,2.0
1,BOG,2022-10-19,2022-10-19 01:15,Frutas & verduras,Verduras,7749572820280257375,Cebolla Cabezona Roja Sin Pelar Tamaño Mixto D...,2899.9498,2916.0,2916.0,2916.0,2916.0,3605.0,3352.65,2.0,1.0
2,BOG,2022-10-19,2022-10-19 01:06,Frutas & verduras,Tubérculos,-8910485076695139605,Papa Criolla Segunda Mixta Kg,2089.4632,NaN,NaN,NaN,NaN,2660.0,2660.00,10.0,NaN
3,BOG,2022-10-20,2022-10-20 14:49,Frutas & verduras,Verduras,-882794291467029550,Pimentón Maduración Mixta Semi (Mediano / Nata...,1876.7926,2500.0,2500.0,2500.0,2500.0,2450.0,2450.00,10.0,2.0
4,BOG,2022-10-19,2022-10-19 19:01,Frutas & verduras,Verduras,5533752035170569793,Cebolla Cabezona Roja Limpia Tamaño Mixto Kg (...,3985.4596,NaN,NaN,NaN,NaN,8112.0,8112.00,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318755,BOG,2023-09-28,2023-09-28 22:36,Frutas & verduras,Tubérculos,7888420588372063556,Papa Criolla Sin Lavar Tamaño Mixto Kg,1941.9497,2222.0,2261.0,2261.0,2300.0,2400.0,2400.00,4.0,2.0
318756,BOG,2023-09-29,2023-09-29 02:55,Frutas & verduras,Verduras,353982459564085093,Rabano Estándar Kg,4594.1957,4000.0,4000.0,4000.0,4000.0,3840.0,3840.00,1.0,1.0
318757,BOG,2023-09-29,2023-09-29 14:51,Frutas & verduras,Tubérculos,-3386165639547488916,Papa Criolla Lavada Semi (Mediana) Kg,3723.2401,3372.0,3372.0,3372.0,3372.0,3260.0,3260.00,2.5,1.0
318758,BOG,2023-09-21,2023-09-21 03:19,Frutas & verduras,Verduras,-1073190491589771159,Champiñón Blanco Paris 500 gr Bandeja,12616.8519,12500.0,12750.0,12750.0,13000.0,12790.0,11766.80,1.0,2.0


In [308]:
new_data = pd.read_csv('data/Frubana_2023_10_09.csv')
new_data['order_submited_datetime'] = pd.to_datetime(new_data['order_submited_datetime'])
new_data['order_submited_datetime'] = new_data['order_submited_datetime'].dt.tz_localize(pytz.utc).dt.tz_convert(colombia_tz)
new_data['order_submited_date'] = new_data['order_submited_datetime'].dt.date.astype(str)

new_data = new_data[new_data['order_submited_date']>'2023-09-29']
new_data


,region_code,order_submited_date,order_submited_datetime,parent_category,category,product_id,product_name,cost,benchmark,median_bench,avg_bench,max_bench,gross_price,net_price,quantity,product_step_unit,benchmark_count
18910,BOG,2023-09-30,2023-09-30 08:26:00-05:00,Frutas & verduras,Verduras,7749572820280257375,Cebolla Cabezona Roja Sin Pelar Tamaño Mixto D...,4345.9175,3750.0,3750.0,3750.0,3750.0,3600.84,3410.00,10.0,5.0,1.0
18911,BOG,2023-09-30,2023-09-30 12:22:00-05:00,Frutas & verduras,Tubérculos,-3386165639547488916,Papa Criolla Lavada Semi (Mediana) Kg,3069.9237,3571.0,3571.0,3571.0,3571.0,3450.00,3450.00,5.0,2.5,1.0
18912,BOG,2023-09-30,2023-09-30 16:12:00-05:00,Frutas & verduras,Verduras,-6536998881607504388,Apio Estándar (Grande) Atado,615.0477,1333.0,1333.0,1333.0,1333.0,1380.00,1342.74,1.0,1.0,1.0
18913,BOG,2023-10-01,2023-10-01 11:09:00-05:00,Frutas & verduras,Verduras,-6536998881607504388,Apio Estándar (Grande) Atado,NaN,1333.0,1333.0,1333.0,1333.0,1260.00,1225.98,3.0,3.0,1.0
18914,BOG,2023-10-01,2023-10-01 17:59:00-05:00,Frutas & verduras,Verduras,353982459564085093,Rabano Estándar Kg,NaN,4000.0,4000.0,4000.0,4000.0,3840.00,3840.00,0.5,0.5,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322765,BOG,2023-10-08,2023-10-08 17:21:00-05:00,Frutas & verduras,Verduras,-6536998881607504388,Apio Estándar (Grande) Atado,597.9026,1333.0,1333.0,1333.0,1333.0,1540.00,1540.00,1.0,1.0,1.0
322766,BOG,2023-10-08,2023-10-08 18:25:00-05:00,Frutas & verduras,Verduras,-6536998881607504388,Apio Estándar (Grande) Atado,597.9026,1333.0,1333.0,1333.0,1333.0,1410.00,1371.93,9.0,3.0,1.0
322767,BOG,2023-10-08,2023-10-08 19:25:00-05:00,Frutas & verduras,Verduras,353982459564085093,Rabano Estándar Kg,3064.8017,NaN,NaN,NaN,NaN,3840.00,3840.00,2.0,0.5,NaN
322768,BOG,2023-10-08,2023-10-08 20:02:00-05:00,Frutas & verduras,Tubérculos,7888420588372063556,Papa Criolla Sin Lavar Tamaño Mixto Kg,1899.3450,2667.0,2667.0,2667.0,2667.0,2260.00,2260.00,4.0,4.0,1.0


In [305]:
common_columns = data.columns.intersection(historical.columns)
merged_df = pd.concat([historical[common_columns], data[common_columns]])
merged_df


,region_code,order_submited_date,order_submited_datetime,parent_category,category,product_id,product_name,cost,benchmark,median_bench,avg_bench,max_bench,gross_price,net_price,quantity,benchmark_count
0,BOG,2022-10-18,2022-10-18 20:47,Frutas & verduras,Verduras,-6536998881607504388,Apio Estándar (Grande) Atado,901.3589,1000.0,1100.0,1100.0,1200.0,1440.0,1440.00,1.0,2.0
1,BOG,2022-10-19,2022-10-19 01:15,Frutas & verduras,Verduras,7749572820280257375,Cebolla Cabezona Roja Sin Pelar Tamaño Mixto D...,2899.9498,2916.0,2916.0,2916.0,2916.0,3605.0,3352.65,2.0,1.0
2,BOG,2022-10-19,2022-10-19 01:06,Frutas & verduras,Tubérculos,-8910485076695139605,Papa Criolla Segunda Mixta Kg,2089.4632,NaN,NaN,NaN,NaN,2660.0,2660.00,10.0,NaN
3,BOG,2022-10-20,2022-10-20 14:49,Frutas & verduras,Verduras,-882794291467029550,Pimentón Maduración Mixta Semi (Mediano / Nata...,1876.7926,2500.0,2500.0,2500.0,2500.0,2450.0,2450.00,10.0,2.0
4,BOG,2022-10-19,2022-10-19 19:01,Frutas & verduras,Verduras,5533752035170569793,Cebolla Cabezona Roja Limpia Tamaño Mixto Kg (...,3985.4596,NaN,NaN,NaN,NaN,8112.0,8112.00,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322765,BOG,2023-10-08,2023-10-08 22:21,Frutas & verduras,Verduras,-6536998881607504388,Apio Estándar (Grande) Atado,597.9026,1333.0,1333.0,1333.0,1333.0,1540.0,1540.00,1.0,1.0
322766,BOG,2023-10-08,2023-10-08 23:25,Frutas & verduras,Verduras,-6536998881607504388,Apio Estándar (Grande) Atado,597.9026,1333.0,1333.0,1333.0,1333.0,1410.0,1371.93,9.0,1.0
322767,BOG,2023-10-09,2023-10-09 00:25,Frutas & verduras,Verduras,353982459564085093,Rabano Estándar Kg,3064.8017,NaN,NaN,NaN,NaN,3840.0,3840.00,2.0,NaN
322768,BOG,2023-10-09,2023-10-09 01:02,Frutas & verduras,Tubérculos,7888420588372063556,Papa Criolla Sin Lavar Tamaño Mixto Kg,1899.3450,2667.0,2667.0,2667.0,2667.0,2260.0,2260.00,4.0,1.0


<!-- 324544 rows × 16 columns -->
<!-- 5784 rows × 17 columns -->
<!-- 318760 rows × 16 columns -->
dasa


In [212]:
data = pd.read_csv('data/created/Catalan_2023-10-17.csv')
data[data['product_name']=='Apio Estándar (Grande) Atado'].sort_values(['order_submited_date','order_submited_datetime'])

,Unnamed: 0,region_code,order_submited_date,order_submited_datetime,parent_category,category,product_id,product_name,cost,benchmark,median_bench,avg_bench,max_bench,gross_price,net_price,quantity,benchmark_count
0,70286,BOG,2021-09-30,2021-09-29 19:06:00-05:00,Frutas & verduras,Verduras,-6536998881607504388,Apio Estándar (Grande) Atado,NaN,2000.0,2000.0,2000.0,2000.0,2316.0,1968.6,1.0,1.0
8,202542,BOG,2021-09-30,2021-09-29 19:11:00-05:00,Frutas & verduras,Verduras,-6536998881607504388,Apio Estándar (Grande) Atado,NaN,2000.0,2000.0,2000.0,2000.0,1988.0,1988.0,3.0,1.0
11,279098,BOG,2021-09-30,2021-09-29 19:13:00-05:00,Frutas & verduras,Verduras,-6536998881607504388,Apio Estándar (Grande) Atado,NaN,2000.0,2000.0,2000.0,2000.0,2316.0,1968.6,1.0,1.0
19,248353,BOG,2021-09-30,2021-09-29 19:18:00-05:00,Frutas & verduras,Verduras,-6536998881607504388,Apio Estándar (Grande) Atado,NaN,2000.0,2000.0,2000.0,2000.0,2316.0,2316.0,1.0,1.0
25,35027,BOG,2021-09-30,2021-09-29 19:21:00-05:00,Frutas & verduras,Verduras,-6536998881607504388,Apio Estándar (Grande) Atado,NaN,2000.0,2000.0,2000.0,2000.0,2316.0,2316.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319788,707,BOG,2023-10-17,2023-10-17 12:46:00-05:00,Frutas & verduras,Verduras,-6536998881607504388,Apio Estándar (Grande) Atado,615.6391,1200.0,1200.0,1200.0,1200.0,1100.0,1100.0,5.0,2.0
319789,925,BOG,2023-10-17,2023-10-17 12:46:00-05:00,Frutas & verduras,Verduras,-6536998881607504388,Apio Estándar (Grande) Atado,615.6391,1200.0,1200.0,1200.0,1200.0,1500.0,1459.5,1.0,2.0
319796,423,BOG,2023-10-17,2023-10-17 12:50:00-05:00,Frutas & verduras,Verduras,-6536998881607504388,Apio Estándar (Grande) Atado,615.6391,1200.0,1200.0,1200.0,1200.0,1430.0,1430.0,1.0,2.0
319800,560,BOG,2023-10-17,2023-10-17 12:52:00-05:00,Frutas & verduras,Verduras,-6536998881607504388,Apio Estándar (Grande) Atado,615.6391,1200.0,1200.0,1200.0,1200.0,1310.0,1310.0,2.0,2.0


In [33]:
x = pd.read_csv('temp1.csv')
x = x.sort_values(['date','hour'])
x

/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_99804/175337733.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  x = pd.read_csv('temp1.csv')


,Unnamed: 0,date,hour,net_price,quantity,cost,avg_bench,product_id,product_name,gross_price,...,quantity_ewm_alpha_05_lag_30,quantity_ewm_alpha_05_lag_60,rolling_avg_quantity_30_lag2,rolling_avg_quantity_15_lag2,rolling_avg_quantity_10_lag2,rolling_avg_quantity_7_lag2,quantity_daily_lag2,trend_daily_lag2,seasonal_daily_lag2,residual_daily_lag2
24,0,2021-09-29,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25,1,2021-09-29,1,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26,2,2021-09-29,2,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27,3,2021-09-29,3,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28,4,2021-09-29,4,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,38156,2023-10-17,19,1500.0,0.0,615.0477,1300.0,-6.536999e+18,Apio Estándar (Grande) Atado,1500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20,38157,2023-10-17,20,1500.0,0.0,615.0477,1300.0,-6.536999e+18,Apio Estándar (Grande) Atado,1500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21,38158,2023-10-17,21,1500.0,0.0,615.0477,1300.0,-6.536999e+18,Apio Estándar (Grande) Atado,1500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22,38159,2023-10-17,22,1500.0,0.0,615.0477,1300.0,-6.536999e+18,Apio Estándar (Grande) Atado,1500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [280]:
df = pd.read_csv('data/created/Catalan_2023-10-17.csv')
temp = df[df['product_name']=='Champiñón Blanco Paris 1Kg Bandeja']
daily_quantity = temp.groupby('order_submited_date').agg({'quantity': 'sum', 'net_price': 'mean'}).reset_index()
daily_quantity.sort_values('order_submited_date').tail(9+7)
# temp

,order_submited_date,quantity,net_price
704,2023-10-02,92.0,24232.242564
705,2023-10-03,28.0,23035.852500
706,2023-10-04,77.0,20407.343158
707,2023-10-05,52.0,22080.000000
708,2023-10-06,36.0,22009.422857
709,2023-10-07,47.0,22080.000000
710,2023-10-08,34.0,22080.000000
711,2023-10-09,87.0,21883.793171
712,2023-10-10,27.0,21859.583333
713,2023-10-11,72.0,18141.377778


In [96]:
data = pd.read_csv('temp.csv')
data[[
    'date', 'net_price', 'net_price_day_0', 'net_price_day_1', 'net_price_day_2',
    'net_price_day_3', 'net_price_day_4', 'net_price_day_5', 'net_price_day_6',
    'hour', 'day_of_week_n', 'inflation', 'trend_daily_lag2', 'quantity'
]][-96:-72]

,date,net_price,net_price_day_0,net_price_day_1,net_price_day_2,net_price_day_3,net_price_day_4,net_price_day_5,net_price_day_6,hour,day_of_week_n,inflation,trend_daily_lag2,quantity
18029,2023-10-12,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0,3,10.79,65.714286,0.0
18030,2023-10-12,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,1,3,10.79,65.714286,0.0
18031,2023-10-12,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,2,3,10.79,65.714286,0.0
18032,2023-10-12,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,3,3,10.79,65.714286,0.0
18033,2023-10-12,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,4,3,10.79,65.714286,0.0
18034,2023-10-12,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,5,3,10.79,65.714286,0.0
18035,2023-10-12,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,6,3,10.79,65.714286,0.0
18036,2023-10-12,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,7,3,10.79,65.714286,0.0
18037,2023-10-12,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,8,3,10.79,65.714286,0.0
18038,2023-10-12,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,9,3,10.79,65.714286,0.0


# Fixing

## convert before joining both, just take last day from latest data

In [698]:
import pandas as pd

# Load the dataframes
df_17 = pd.read_csv('data/Frubana_2023_10_17.csv')
df_16 = pd.read_csv('data/Frubana_2023_10_16.csv')


# Display the first few rows of each dataframe
df_17_head = df_17.head()
df_16_head = df_16.head()

colombia_tz = pytz.timezone('America/Bogota')
    # Convert the 'order_submited_datetime' column to Colombia timezone
df_17['order_submited_datetime'] = pd.to_datetime(df_17['order_submited_datetime'])
df_17['order_submited_datetime'] = df_17['order_submited_datetime'].dt.tz_localize(pytz.utc).dt.tz_convert(colombia_tz)
df_17['order_submited_date'] = df_17['order_submited_datetime'].dt.date.astype(str)
df_17 = df_17[df_17['order_submited_date']<'2023-10-17']
df_17 = df_17[df_17['order_submited_date']>'2023-10-15']


df_16['order_submited_datetime'] = pd.to_datetime(df_16['order_submited_datetime'])
df_16['order_submited_datetime'] = df_16['order_submited_datetime'].dt.tz_localize(pytz.utc).dt.tz_convert(colombia_tz)
df_16['order_submited_date'] = df_16['order_submited_datetime'].dt.date.astype(str)
df_16 = df_16[df_16['order_submited_date']<'2023-10-16']

# df_16 = df_16[df_16['order_submited_date']<'2023-10-16']
# df_17 = df_17[df_17['order_submited_date']<'2023-10-17']


df_17_head, df_16_head

(  region_code order_submited_date order_submited_datetime    parent_category  \
 0         BOG          2023-10-16        2023-10-16 00:32  Frutas & verduras   
 1         BOG          2023-10-16        2023-10-16 12:25  Frutas & verduras   
 2         BOG          2023-10-16        2023-10-16 17:21  Frutas & verduras   
 3         BOG          2023-10-16        2023-10-16 19:35  Frutas & verduras   
 4         BOG          2023-10-16        2023-10-16 20:04  Frutas & verduras   
 
    category          product_id  \
 0  Verduras  353982459564085093   
 1  Verduras  353982459564085093   
 2  Verduras -882794291467029550   
 3  Verduras  353982459564085093   
 4  Verduras -882794291467029550   
 
                                         product_name       cost  benchmark  \
 0                                 Rabano Estándar Kg  3354.9327     4000.0   
 1                                 Rabano Estándar Kg  3354.9327     4000.0   
 2  Pimentón Maduración Mixta Semi (Mediano / Nata...  21

In [577]:
df_17.columns

Index(['region_code', 'order_submited_date', 'order_submited_datetime',
       'parent_category', 'category', 'product_id', 'product_name', 'cost',
       'benchmark', 'median_bench', 'avg_bench', 'max_bench', 'gross_price',
       'net_price', 'quantity', 'product_step_unit', 'benchmark_count'],
      dtype='object')

In [746]:
# Concatenate the dataframes
combined_df = pd.concat([df_17, df_16], ignore_index=True)

# Drop duplicates based on the mentioned columns
cleaned_df = combined_df
# .drop_duplicates(subset=[  'order_submited_date','order_submited_datetime',
#        'product_id', 'product_name',
       
#        'net_price', 
#        'quantity', 
#        # 'product_step_unit',
#          ])
# product_step_unit

# Display the shape of the original combined dataframe and the cleaned dataframe
original_shape = combined_df.shape
cleaned_shape = cleaned_df.shape

original_shape, cleaned_shape


((325017, 17), (325017, 17))

In [747]:
temp = cleaned_df.sort_values('order_submited_datetime')
temp = temp[temp['product_name']=='Papa Criolla Segunda Mixta Kg']
# temp[temp['order_submited_date']=='2023-10-15']
colombia_tz = pytz.timezone('America/Bogota')
    # Convert the 'order_submited_datetime' column to Colombia timezone
# temp['order_submited_datetime'] = pd.to_datetime(temp['order_submited_datetime'])
# temp['order_submited_datetime'] = temp['order_submited_datetime'].dt.tz_localize(pytz.utc).dt.tz_convert(colombia_tz)
# temp['order_submited_date'] = temp['order_submited_datetime'].dt.date.astype(str)
# temp[temp['order_submited_date']=='2023-10-15']
day = temp[temp['order_submited_date']=='2023-10-16']
day['quantity'].sum()
# day['quantity'].sum()

30.0

* 2023-10-16
* Apio Estándar (Grande) Atado - actual 473, found 470
* Cebolla Cabezona Roja Sin Pelar Tamaño Mixto Desde 2Kg, actual 418, found 411
* Pimentón Maduración Mixta Semi (Mediano / Nataly) Desde 2Kg, acual 406, found 404
* Champiñón Blanco Paris 1Kg Bandeja actual 50, found 49,
* Papa Criolla Segunda Mixta Kg actua40, found 30
* Fresa Pequeño Kg actual 72, found 77
* Champiñón Blanco Paris 500 gr Bandeja actual 27,found 28 

In [459]:
temp = df_16.sort_values('order_submited_datetime')
temp = temp[temp['product_name']=='Papa Sabanera Semi (Mediana) Kg']
# temp[temp['order_submited_date']=='2023-10-15']
colombia_tz = pytz.timezone('America/Bogota')
    # Convert the 'order_submited_datetime' column to Colombia timezone
# temp['order_submited_datetime'] = pd.to_datetime(temp['order_submited_datetime'])
# temp['order_submited_datetime'] = temp['order_submited_datetime'].dt.tz_localize(pytz.utc).dt.tz_convert(colombia_tz)
# temp['order_submited_date'] = temp['order_submited_datetime'].dt.date.astype(str)
# temp[temp['order_submited_date']=='2023-10-15']
day = temp[temp['order_submited_date']=='2023-10-16']
day
# day['quantity'].sum()

,region_code,order_submited_date,order_submited_datetime,parent_category,category,product_id,product_name,cost,benchmark,median_bench,avg_bench,max_bench,gross_price,net_price,quantity,product_step_unit,benchmark_count
125552,BOG,2023-10-16,2023-10-16 02:34,Frutas & verduras,Tubérculos,-368218666129834277,Papa Sabanera Semi (Mediana) Kg,4000.6246,5626.0,5626.0,5626.0,5626.0,5490.0,5490.0,2.5,2.5,1.0


In [690]:
temp = df_17.sort_values('order_submited_datetime')
temp = temp[temp['product_name']=='Fresa Pequeño Kg']
# temp[temp['order_submited_date']=='2023-10-15']
colombia_tz = pytz.timezone('America/Bogota')
    # Convert the 'order_submited_datetime' column to Colombia timezone
# temp['order_submited_datetime'] = pd.to_datetime(temp['order_submited_datetime'])
# temp['order_submited_datetime'] = temp['order_submited_datetime'].dt.tz_localize(pytz.utc).dt.tz_convert(colombia_tz)
# temp['order_submited_date'] = temp['order_submited_datetime'].dt.date.astype(str)
# temp[temp['order_submited_date']=='2023-10-15']
day = temp[temp['order_submited_date']=='2023-10-16']
day
# day['quantity'].sum()

,region_code,order_submited_date,order_submited_datetime,parent_category,category,product_id,product_name,cost,benchmark,median_bench,avg_bench,max_bench,gross_price,net_price,quantity,product_step_unit,benchmark_count
360,BOG,2023-10-16,2023-10-16 01:01,Frutas & verduras,Frutas,-4543321951091465013,Fresa Pequeño Kg,3785.5838,5714.0,5714.0,5714.0,5714.0,6570.0,6570.0,3.0,3.0,1
728,BOG,2023-10-16,2023-10-16 13:35,Frutas & verduras,Frutas,-4543321951091465013,Fresa Pequeño Kg,3785.5838,5714.0,5714.0,5714.0,5714.0,6260.0,6260.0,5.0,5.0,1
1038,BOG,2023-10-16,2023-10-16 15:30,Frutas & verduras,Frutas,-4543321951091465013,Fresa Pequeño Kg,3785.5838,5714.0,5714.0,5714.0,5714.0,6570.0,6570.0,3.0,3.0,1
186,BOG,2023-10-16,2023-10-16 15:35,Frutas & verduras,Frutas,-4543321951091465013,Fresa Pequeño Kg,3785.5838,5714.0,5714.0,5714.0,5714.0,6260.0,6260.0,5.0,5.0,1
738,BOG,2023-10-16,2023-10-16 15:47,Frutas & verduras,Frutas,-4543321951091465013,Fresa Pequeño Kg,3785.5838,5714.0,5714.0,5714.0,5714.0,6570.0,6570.0,3.0,3.0,1
229,BOG,2023-10-16,2023-10-16 15:59,Frutas & verduras,Frutas,-4543321951091465013,Fresa Pequeño Kg,3785.5838,5714.0,5714.0,5714.0,5714.0,6570.0,6570.0,3.0,3.0,1
851,BOG,2023-10-16,2023-10-16 17:02,Frutas & verduras,Frutas,-4543321951091465013,Fresa Pequeño Kg,3785.5838,5714.0,5714.0,5714.0,5714.0,6570.0,6570.0,3.0,3.0,1
109,BOG,2023-10-16,2023-10-16 17:36,Frutas & verduras,Frutas,-4543321951091465013,Fresa Pequeño Kg,3785.5838,5714.0,5714.0,5714.0,5714.0,6570.0,6570.0,3.0,3.0,1
497,BOG,2023-10-16,2023-10-16 19:35,Frutas & verduras,Frutas,-4543321951091465013,Fresa Pequeño Kg,3785.5838,5714.0,5714.0,5714.0,5714.0,6570.0,6570.0,3.0,3.0,1
338,BOG,2023-10-16,2023-10-16 19:56,Frutas & verduras,Frutas,-4543321951091465013,Fresa Pequeño Kg,3785.5838,5714.0,5714.0,5714.0,5714.0,6570.0,6570.0,3.0,3.0,1


In [475]:
temp = df_17.sort_values('order_submited_datetime')
temp = temp[temp['product_name']=='Cebolla Cabezona Roja Sin Pelar Tamaño Mixto Desde 2Kg']
# temp[temp['order_submited_date']=='2023-10-15']
colombia_tz = pytz.timezone('America/Bogota')
    # Convert the 'order_submited_datetime' column to Colombia timezone
# temp['order_submited_datetime'] = pd.to_datetime(temp['order_submited_datetime'])
# temp['order_submited_datetime'] = temp['order_submited_datetime'].dt.tz_localize(pytz.utc).dt.tz_convert(colombia_tz)
# temp['order_submited_date'] = temp['order_submited_datetime'].dt.date.astype(str)
# temp[temp['order_submited_date']=='2023-10-15']
day = temp[temp['order_submited_date']=='2023-10-16']
# day
day['quantity'].sum()

347.0

In [483]:
temp1 = df_17[df_17['order_submited_date']=='2023-10-16'].sort_values('order_submited_datetime').head(175)

In [484]:
temp2 = df_16[df_16['order_submited_date']=='2023-10-16'].sort_values('order_submited_datetime')

In [485]:
temp1.to_csv('temp1.csv')
temp2.to_csv('temp2.csv')

In [464]:
temp = cleaned_df.sort_values('order_submited_datetime')
temp = temp[temp['product_name']=='Papa Sabanera Semi (Mediana) Kg']
# temp[temp['order_submited_date']=='2023-10-15']
colombia_tz = pytz.timezone('America/Bogota')
    # Convert the 'order_submited_datetime' column to Colombia timezone
temp['order_submited_datetime'] = pd.to_datetime(temp['order_submited_datetime'])
temp['order_submited_datetime'] = temp['order_submited_datetime'].dt.tz_localize(pytz.utc).dt.tz_convert(colombia_tz)
temp['order_submited_date'] = temp['order_submited_datetime'].dt.date.astype(str)
# temp[temp['order_submited_date']=='2023-10-15']
day = temp[temp['order_submited_date']=='2023-10-16']
day
# day['quantity'].sum()

,region_code,order_submited_date,order_submited_datetime,parent_category,category,product_id,product_name,cost,benchmark,median_bench,avg_bench,max_bench,gross_price,net_price,quantity,product_step_unit,benchmark_count
947,BOG,2023-10-16,2023-10-16 11:37:00-05:00,Frutas & verduras,Tubérculos,-368218666129834277,Papa Sabanera Semi (Mediana) Kg,4103.355,5626.0,5626.0,5626.0,5626.0,5490.0,5490.0,5.0,2.5,1.0
415,BOG,2023-10-16,2023-10-16 12:16:00-05:00,Frutas & verduras,Tubérculos,-368218666129834277,Papa Sabanera Semi (Mediana) Kg,4103.355,5626.0,5626.0,5626.0,5626.0,5490.0,5490.0,2.5,2.5,1.0
496,BOG,2023-10-16,2023-10-16 14:34:00-05:00,Frutas & verduras,Tubérculos,-368218666129834277,Papa Sabanera Semi (Mediana) Kg,4103.355,5626.0,5626.0,5626.0,5626.0,5490.0,5490.0,2.5,2.5,1.0
101,BOG,2023-10-16,2023-10-16 17:03:00-05:00,Frutas & verduras,Tubérculos,-368218666129834277,Papa Sabanera Semi (Mediana) Kg,4103.355,5626.0,5626.0,5626.0,5626.0,5490.0,5490.0,2.5,2.5,1.0
177,BOG,2023-10-16,2023-10-16 19:37:00-05:00,Frutas & verduras,Tubérculos,-368218666129834277,Papa Sabanera Semi (Mediana) Kg,4103.355,5626.0,5626.0,5626.0,5626.0,5490.0,5490.0,2.5,2.5,2.0


In [327]:
df1= pd.read_csv('data/cr_2023-10-16.csv')
df2 = pd.read_csv('data/created/Catalan_2023-10-17.csv')
df = pd.concat([df1,df2])
df=df.drop_duplicates()
temp = df[df['product_name']=='Apio Estándar (Grande) Atado']
daily_quantity = temp.groupby('order_submited_date').agg({'quantity': 'sum', 'net_price': 'mean'}).reset_index()
daily_quantity.sort_values('order_submited_date').tail(9+7)

,order_submited_date,quantity,net_price
726,2023-10-02,573.0,1291.268387
727,2023-10-03,457.0,1217.358182
728,2023-10-04,418.0,1271.382252
729,2023-10-05,410.0,1411.191125
730,2023-10-06,339.0,1455.012540
731,2023-10-07,99.0,1459.366739
732,2023-10-08,379.0,1438.333741
733,2023-10-09,497.0,1426.646461
734,2023-10-10,393.0,1307.325959
735,2023-10-11,516.0,1220.283013


In [332]:
df

,region_code,order_submited_date,order_submited_datetime,parent_category,category,product_id,product_name,cost,benchmark,median_bench,avg_bench,max_bench,gross_price,net_price,quantity,product_step_unit,benchmark_count
0,BOG,2022-10-19,2022-10-19 12:31,Frutas & verduras,Verduras,5533752035170569793,Cebolla Cabezona Roja Limpia Tamaño Mixto Kg (...,3985.4596,NaN,NaN,NaN,NaN,8112.00,8112.0,1.0,1.0,NaN
1,BOG,2022-10-19,2022-10-19 22:43,Frutas & verduras,Verduras,-6536998881607504388,Apio Estándar (Grande) Atado,898.5938,1000.0,1000.0,1000.0,1000.0,1284.00,1284.0,4.0,1.0,1.0
2,BOG,2022-10-19,2022-10-19 21:29,Frutas & verduras,Verduras,3038455164096626586,Guatila Estándar Kg,1173.2613,NaN,NaN,NaN,NaN,1680.00,1680.0,15.0,1.0,NaN
3,BOG,2022-10-19,2022-10-19 22:57,Frutas & verduras,Verduras,-6536998881607504388,Apio Estándar (Grande) Atado,898.5938,1000.0,1000.0,1000.0,1000.0,1102.00,1102.0,3.0,3.0,1.0
4,BOG,2022-10-19,2022-10-19 14:17,Frutas & verduras,Verduras,-882794291467029550,Pimentón Maduración Mixta Semi (Mediano / Nata...,1880.7317,2500.0,2500.0,2500.0,2500.0,2450.00,2450.0,3.0,1.5,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1039,BOG,2023-10-17,2023-10-17 00:26,Frutas & verduras,Verduras,-882794291467029550,Pimentón Maduración Mixta Semi (Mediano / Nata...,2190.4199,4000.0,4000.0,4000.0,4000.0,3550.00,3550.0,2.0,1.0,2.0
1040,BOG,2023-10-16,2023-10-16 14:06,Frutas & verduras,Verduras,3038455164096626586,Guatila Estándar Kg,1687.3770,2000.0,2000.0,2000.0,2000.0,1880.00,1880.0,1.0,1.0,1.0
1041,BOG,2023-10-17,2023-10-17 12:25,Frutas & verduras,Verduras,5533752035170569793,Cebolla Cabezona Roja Limpia Tamaño Mixto Kg (...,3550.4933,3400.0,3400.0,3400.0,3400.0,3616.68,3425.0,1.0,1.0,2.0
1042,BOG,2023-10-17,2023-10-17 13:50,Frutas & verduras,Tubérculos,-3386165639547488916,Papa Criolla Lavada Semi (Mediana) Kg,2515.4174,3372.0,3372.5,3372.5,3373.0,3600.00,3502.8,5.0,2.5,2.0


In [ ]:
df = pd.read_csv('historical')
temp = df[df['product_name']=='Apio Estándar (Grande) Atado']
daily_quantity = temp.groupby('order_submited_date').agg({'quantity': 'sum', 'net_price': 'mean'}).reset_index()
daily_quantity.sort_values('order_submited_date').tail(9+7)